# Text Main

> This module contains the main Python class for data control: `TextDataMain`

```#| default_exp text_main```

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from datasets import DatasetDict,Dataset,IterableDataset,load_dataset,concatenate_datasets
from pathlib import Path
from tqdm import tqdm
from that_nlp_library.utils import *
from functools import partial
import warnings

In [ ]:
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from importlib.machinery import SourceFileLoader
import os

## Content Transformation, Augmentations, and Tokenization

In [ ]:
#| export
def tokenizer_explain(inp, # Input sentence
                      tokenizer, # Tokenizer (preferably from HuggingFace)
                      split_word=False # Is input `inp` split into list or not
                     ):
    "Display results from tokenizer"
    print('----- Tokenizer Explained -----')
    print('--- Input ---')
    print(inp)
    print()
    print('--- Tokenized results --- ')
    print(tokenizer(inp,is_split_into_words=split_word))
    print()
    tok = tokenizer.encode(inp,is_split_into_words=split_word)
    print('--- Results from tokenizer.convert_ids_to_tokens ---')
    print(tokenizer.convert_ids_to_tokens(tok))
    print()
    print('--- Results from tokenizer.decode --- ')
    print(tokenizer.decode(tok))
    print()

In [ ]:
show_doc(tokenizer_explain)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L20){target="_blank" style="float:right; font-size:smaller"}

### tokenizer_explain

>      tokenizer_explain (inp, tokenizer, split_word=False)

Display results from tokenizer

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| inp |  |  | Input sentence |
| tokenizer |  |  | Tokenizer (preferably from HuggingFace) |
| split_word | bool | False | Is input `inp` split into list or not |

Let's load a tokenizer from EnviBert model. Uncomment the command line below to download necessary files to build this tokenizer

In [ ]:
# !pip install gdown

In [ ]:
# !gdown 14X9fGijA7kdNfe4dM_8gqfxIWtj1Q-hb -O ./envibert_cache --folder

In [ ]:
cache_dir=Path('./envibert_tokenizer')
tokenizer = SourceFileLoader("envibert.tokenizer", 
                             str(cache_dir/'envibert_tokenizer.py')).load_module().RobertaTokenizer(cache_dir)

Note that Envibert tokenizer does not required the input to be tokenized using word_tokenize from UnderTheSea library

In [ ]:
inp = 'Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh'
tokenizer_explain(inp,tokenizer,split_word=False)

----- Tokenizer Explained -----
--- Input ---
Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh

--- Tokenized results --- 
{'input_ids': [0, 857, 1033, 191, 664, 1033, 7366, 2615, 142, 664, 1033, 671, 1355, 2294, 993, 413, 2900, 244, 1019, 827, 24, 40, 647, 773, 549, 119, 511, 1134, 1690, 758, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

--- Results from tokenizer.convert_ids_to_tokens ---
['<s>', '▁Hội', '▁cư', '▁dân', '▁chung', '▁cư', '▁sen', '▁hồng', '-', '▁chung', '▁cư', '▁lot', 'us', '▁sóng', '▁thần', '▁thủ', '▁đức', '.', '▁Thủ', '▁Đức', '▁là', '▁một', '▁huyện', '▁trực', '▁thuộc', '▁thành', '▁phố', '▁Hồ', '▁Chí', '▁Minh', '</s>']

--- Results from tokenizer.decode --- 
<s> ▁Hội ▁cư ▁dân ▁chung ▁cư ▁sen ▁hồng - ▁chung ▁cư ▁lot

In [ ]:
inp = ['hội', 'cư', 'dân', 'chung', 'cư', 'sen', 'hồng', '-', 'chung', 'cư', 'lotus', 'sóng', 'thần', 'thủ', 'đức']
tokenizer_explain(inp,tokenizer,split_word=True)

----- Tokenizer Explained -----
--- Input ---
['hội', 'cư', 'dân', 'chung', 'cư', 'sen', 'hồng', '-', 'chung', 'cư', 'lotus', 'sóng', 'thần', 'thủ', 'đức']

--- Tokenized results --- 
{'input_ids': [0, 227, 1033, 191, 664, 1033, 7366, 2615, 13, 664, 1033, 671, 1355, 2294, 993, 413, 2900, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

--- Results from tokenizer.convert_ids_to_tokens ---
['<s>', '▁hội', '▁cư', '▁dân', '▁chung', '▁cư', '▁sen', '▁hồng', '▁-', '▁chung', '▁cư', '▁lot', 'us', '▁sóng', '▁thần', '▁thủ', '▁đức', '</s>']

--- Results from tokenizer.decode --- 
<s> ▁hội ▁cư ▁dân ▁chung ▁cư ▁sen ▁hồng ▁- ▁chung ▁cư ▁lot us ▁sóng ▁thần ▁thủ ▁đức </s>



Now let's try PhoBert tokenizer. PhoBert tokenizer, unlike Envibert tokenizer, requires input to be word tokenized (using UnderTheSea library)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
inp = apply_vnmese_word_tokenize('hội cư dân chung cư sen hồng - chung cư lotus sóng thần thủ đức')
print(inp)

hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức


In [ ]:
tokenizer_explain(inp,tokenizer)

----- Tokenizer Explained -----
--- Input ---
hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức

--- Tokenized results --- 
{'input_ids': [0, 1093, 1838, 1574, 3330, 2025, 31, 1574, 2029, 4885, 8554, 25625, 7344, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

--- Results from tokenizer.convert_ids_to_tokens ---
['<s>', 'hội', 'cư_dân', 'chung_cư', 'sen', 'hồng', '-', 'chung_cư', 'lo@@', 'tus', 'sóng_thần', 'thủ_@@', 'đức', '</s>']

--- Results from tokenizer.decode --- 
<s> hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức </s>



In [ ]:
#| export
def two_steps_tokenization_explain(inp, # Input sentence
                                   tokenizer, # Tokenizer (preferably from HuggingFace)
                                   content_tfms=[], # A list of text transformations
                                   aug_tfms=[], # A list of text augmentation 
                                  ):
    "Display results form each content transformation, then display results from tokenizer"
    print('----- Text Transformation Explained -----')
    print('--- Raw sentence ---')
    print(inp)
    print('--- Content Transformations (on both train and test) ---')
    content_tfms = val2iterable(content_tfms)
    for tfm in content_tfms:
        print_msg(callable_name(tfm),3)
        inp = tfm(inp)
        print(inp)
    print('--- Augmentations (on train only) ---')
    aug_tfms = val2iterable(aug_tfms)
    for tfm in aug_tfms:
        print_msg(callable_name(tfm),3)
        inp = tfm(inp)
        print(inp)
    print()
    tokenizer_explain(inp,tokenizer)

In [ ]:
show_doc(two_steps_tokenization_explain)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L41){target="_blank" style="float:right; font-size:smaller"}

### two_steps_tokenization_explain

>      two_steps_tokenization_explain (inp, tokenizer, content_tfms=[],
>                                      aug_tfms=[])

Display results form each content transformation, then display results from tokenizer

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| inp |  |  | Input sentence |
| tokenizer |  |  | Tokenizer (preferably from HuggingFace) |
| content_tfms | list | [] | A list of text transformations |
| aug_tfms | list | [] | A list of text augmentation |

Let's load Phobert tokenizer one more time to test out this function

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from underthesea import text_normalize

`apply_vnmese_word_tokenize` also have an option to normalize text

In [ ]:
from functools import partial

In [ ]:
inp = 'Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh'
two_steps_tokenization_explain(inp,tokenizer,content_tfms=[partial(apply_vnmese_word_tokenize,normalize_text=True)])

----- Text Transformation Explained -----
--- Raw sentence ---
Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh
--- Content Transformations (on both train and test) ---
--- apply_vnmese_word_tokenize ---
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh
--- Augmentations (on train only) ---

----- Tokenizer Explained -----
--- Input ---
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh

--- Tokenized results --- 
{'input_ids': [0, 792, 1838, 1574, 3330, 2025, 31, 1574, 2029, 4885, 8554, 25625, 7344, 5, 5043, 8, 16, 149, 2850, 214, 784, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

--- Results from tokenizer.convert_ids_to_tokens ---
['<s>', 'Hội',

Let's add some text augmentations

In [ ]:
import unidecode

In [ ]:
# to remove vietnamese accent
remove_accent = lambda x: unidecode.unidecode(x)

If you want your function to be printed in with a different name:

In [ ]:
remove_accent.__name__ = 'Remove Vietnamese Accent'

In [ ]:
two_steps_tokenization_explain(inp,tokenizer,
                               content_tfms=[partial(apply_vnmese_word_tokenize,normalize_text=True)],
                               aug_tfms=[remove_accent]
                              )

----- Text Transformation Explained -----
--- Raw sentence ---
Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh
--- Content Transformations (on both train and test) ---
--- apply_vnmese_word_tokenize ---
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh
--- Augmentations (on train only) ---
--- Remove Vietnamese Accent ---
Hoi cu_dan chung_cu sen hong - chung_cu lotus song_than thu_duc . Thu_Duc la mot huyen truc_thuoc thanh_pho Ho_Chi_Minh

----- Tokenizer Explained -----
--- Input ---
Hoi cu_dan chung_cu sen hong - chung_cu lotus song_than thu_duc . Thu_Duc la mot huyen truc_thuoc thanh_pho Ho_Chi_Minh

--- Tokenized results --- 
{'input_ids': [0, 3021, 1111, 56549, 17386, 22975, 13689, 3330, 27037, 31, 22975, 13689, 2029, 4885, 3227, 9380, 1510, 21605, 6190, 1894, 5, 5770, 4098, 1894, 2644, 3773, 1204, 18951, 2052, 10242, 9835, 1881, 22899, 

You can even be creative with your augmentation functions; let's say you only want your augmentation to be applied 50% of the time:

In [ ]:
import random

In [ ]:
random.seed(2) # for reproducibility

In [ ]:
remove_accent = lambda x: unidecode.unidecode(x) if random.random()<0.5 else x
remove_accent.__name__ = 'Remove Vietnamese Accent with 0.5 prob'

In [ ]:
two_steps_tokenization_explain(inp,tokenizer,
                               content_tfms=[partial(apply_vnmese_word_tokenize,normalize_text=True)],
                               aug_tfms=[remove_accent]
                              )

----- Text Transformation Explained -----
--- Raw sentence ---
Hội cư dân   chung cư sen hồng- chung cư    lotus sóng thần thủ đức. Thủ Đức là một huyện trực thuộc thành phố Hồ Chí Minh
--- Content Transformations (on both train and test) ---
--- apply_vnmese_word_tokenize ---
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh
--- Augmentations (on train only) ---
--- Remove Vietnamese Accent with 0.5 prob ---
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh

----- Tokenizer Explained -----
--- Input ---
Hội cư_dân chung_cư sen hồng - chung_cư lotus sóng_thần thủ_đức . Thủ_Đức là một huyện trực_thuộc thành_phố Hồ_Chí_Minh

--- Tokenized results --- 
{'input_ids': [0, 792, 1838, 1574, 3330, 2025, 31, 1574, 2029, 4885, 8554, 25625, 7344, 5, 5043, 8, 16, 149, 2850, 214, 784, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

There are more examples of interesting augmentation [here](https://anhquan0412.github.io/that-nlp-library/text_augmentation.html)

## Tokenize Function

In [ ]:
#| export
def tokenize_function(examples:dict,
                      tok,
                      text_name,
                      max_length=None,
                      is_split_into_words=False):
    if max_length is None:
        # pad to model's default max sequence length
        return tok(examples[text_name], padding="max_length", truncation=True,is_split_into_words=is_split_into_words)
    if isinstance(max_length,int) and max_length>0:
        # pad to max length of the current batch, and start truncating at max_length
        return tok(examples[text_name], padding=True, max_length=max_length,truncation=True,is_split_into_words=is_split_into_words)
    
    # no padding (still truncate at model's default max sequence length)
    return tok(examples[text_name], truncation=True,is_split_into_words=is_split_into_words)

In [ ]:
show_doc(tokenize_function)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#L58){target="_blank" style="float:right; font-size:smaller"}

### tokenize_function

>      tokenize_function (examples:dict, tok, text_name, max_length=None,
>                         is_split_into_words=False)

Since I am processing Vietnamese text, I will use EnViBert's tokenizer. Envibert is a RoBERTa model for Vietnamese and English. This RoBERTa version is trained by using 100GB of text (50GB of Vietnamese and 50GB of English). For more information: [https://huggingface.co/nguyenvulebinh/envibert](https://huggingface.co/nguyenvulebinh/envibert)

In [ ]:
# https://huggingface.co/nguyenvulebinh/envibert
cache_dir=Path('./envibert_tokenizer')
tokenizer = SourceFileLoader("envibert.tokenizer", 
                             str(cache_dir/'envibert_tokenizer.py')).load_module().RobertaTokenizer(cache_dir)

In [ ]:
examples={
    'text':[
         'hội cư dân chung cư sen hồng - chung cư lotus sóng thần thủ đức',
         'This is the recommended way to make a Python package importable from anywhere',
         'hội cần mở thẻ tín dụng tại hà nội, đà nẵng, tp. hồ chí minh',
         "biti's cao lãnh - đồng tháp",
         'chợ phòng trọ + việc làm...khu lĩnh nam - vĩnh hưng - mai động (hoàng mai)'
          ],
}

In [ ]:
results = tokenize_function(examples,tokenizer,text_name='text',max_length=512)

In [ ]:
results

{'input_ids': [[0, 227, 1033, 191, 664, 1033, 7366, 2615, 13, 664, 1033, 671, 1355, 2294, 993, 413, 2900, 2, 1, 1, 1, 1, 1, 1, 1], [0, 116, 14, 6, 3169, 270, 9, 364, 10, 23963, 5360, 15930, 2003, 51, 5906, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 227, 256, 778, 2600, 1074, 144, 76, 5489, 613, 57339, 4820, 27666, 57339, 21422, 244, 872, 635, 841, 2, 1, 1, 1, 1, 1], [0, 880, 592, 427, 162, 171, 906, 13, 122, 6553, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 2299, 315, 5995, 1349, 99, 83, 55025, 244, 6356, 1114, 1213, 1163, 13, 8233, 11051, 13, 3335, 109, 28, 11695, 13377, 3335, 3, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'att

In [ ]:
print(tokenizer.convert_ids_to_tokens(results['input_ids'][0]))

['<s>', '▁hội', '▁cư', '▁dân', '▁chung', '▁cư', '▁sen', '▁hồng', '▁-', '▁chung', '▁cư', '▁lot', 'us', '▁sóng', '▁thần', '▁thủ', '▁đức', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


You can change max_length (which allow truncation when sentence length is higher than max_length) 

In [ ]:
results = tokenize_function(examples,tokenizer,text_name='text',max_length=5)

In [ ]:
results

{'input_ids': [[0, 227, 1033, 191, 2], [0, 116, 14, 6, 2], [0, 227, 256, 778, 2], [0, 880, 592, 427, 2], [0, 2299, 315, 5995, 2]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

## Metadatas Processing 

In [ ]:
#| export
def concat_metadatas(dset:dict, # HuggingFace Dataset
                     main_text, # Text feature name
                     metadatas, # Metadata (or a list of metadatas)
                     process_metas=True, # Whether apply simple metadata processing, i.e. space strip and lowercase
                     sep='.', # separator for contatenating to main_text
                     is_batched=True, # whether batching is applied
                    ):
    """
    Extract, process (optional) and concatenate metadatas to the front of text
    """
    results={main_text:dset[main_text]}
    for m in metadatas:
        m_data = dset[m]
        if process_metas:
            # just strip and lowercase
            m_data = [none2emptystr(v).strip().lower() for v in m_data] if is_batched else nan2emptystr(m_data).strip().lower()
        results[m]=m_data
        if is_batched:
            results[main_text] = [f'{m_data[i]} {sep} {results[main_text][i]}' for i in range(len(m_data))]
        else:
            results[main_text] = f'{m_data} {sep} {results[main_text]}'
    return results

In [ ]:
show_doc(concat_metadatas)

---

### concat_metadatas

>      concat_metadatas (dset:dict, main_text, metadatas, process_metas=True,
>                        sep='.', is_batched=True)

Extract, process (optional) and concatenate metadatas to the front of text

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| dset | dict |  | HuggingFace Dataset |
| main_text |  |  | Text feature name |
| metadatas |  |  | Metadata (or a list of metadatas) |
| process_metas | bool | True | Whether apply simple metadata processing, i.e. space strip and lowercase |
| sep | str | . | separator for contatenating to main_text |
| is_batched | bool | True | whether batching is applied |

## Class TextDataController

In [ ]:
#| export
class TextDataController():
    def __init__(self,
                 inp, # HuggingFace Dataset or DatasetDict
                 main_text:str, # Name of the main text column
                 label_names=None, # Names of the label (dependent variable) columns
                 class_names_predefined=None, # List of names associated with the labels (same index order)
                 filter_dict={}, # A dictionary: {feature: filtering_function_based_on_the_feature}
                 metadatas=[], # Names of the metadata columns
                 process_metas=True, # Whether to do simple text processing on the chosen metadatas
                 content_transformations=[], # A list of text transformations
                 val_ratio:list|float|None=0.2, # Ratio of data for validation set. If given a list, validation set will be chosen based on indices in this list
                 stratify_cols=[], # Column(s) needed to do stratified shuffle split
                 upsampling_list={}, # A list of tuple. Each tuple: (feature,upsampling_function_based_on_the_feature)
                 content_augmentations=[], # A list of text augmentations
                 seed=None, # Random seed
                 is_batched=True, # Whether to perform operations in batch
                 batch_size=1000, # Batch size, for when is_batched is True
                 num_proc=4, # Number of process for multiprocessing
                 cols_to_keep=None, # Columns to keep after all processings
                 buffer_size=10000, # For shuffling data
                 num_shards=64, # Number of shards
                ):
            
        self.main_text = main_text
        self.metadatas = val2iterable(metadatas)
        self.process_metas = process_metas
        self.label_names = val2iterable(label_names) if label_names is not None else None
        self.label_lists = class_names_predefined
        self.filter_dict = filter_dict
        self.content_tfms = val2iterable(content_transformations)
        self.upsampling_list = upsampling_list
        self.aug_tfms = val2iterable(content_augmentations)
        self.val_ratio = val_ratio
        self.stratify_cols = val2iterable(stratify_cols)
        self.seed = seed
        self.is_batched = is_batched
        self.batch_size = batch_size
        self.num_proc = num_proc
        self.is_streamed = False
        self.cols_to_keep = cols_to_keep
        self.buffer_size = buffer_size
        self.num_shards = num_shards
        self.ddict_rest = DatasetDict()
        
        if hasattr(inp,'keys'):
            if 'train' in inp.keys(): # is datasetdict
                self.ddict_rest = inp
                self.dset = self.ddict_rest.pop('train')
            else:
                raise ValueError('The given DatasetDict has no "train" split')
        else: # is dataset
            self.dset = inp
        if isinstance(self.dset,IterableDataset):
            self.is_streamed=True
        self.all_cols = self.dset.column_names
        
        if self.is_streamed and self.label_names is not None and self.label_lists is None:
            raise ValueError('All class labels must be provided when streaming')
        
        if self.is_streamed and len(self.upsampling_list):
            warnings.warn("Upsampling requires dataset concatenation, which can be extremely slow (x2) for streamed dataset")
            
        self._processed_call=False
        self.is_multilabel=False
        self.is_multihead=False
        
        
            
    @classmethod
    def from_csv(cls,file_path,**kwargs):
        file_path = Path(file_path)
        ds = load_dataset(str(file_path.parent),
                                  data_files=file_path.name,
                                  split='train')
        return TextDataController(ds,**kwargs)
        
    
    @classmethod
    def from_df(cls,df,validate=True,**kwargs):
        if validate:
            check_input_validation(df)
        ds = Dataset.from_pandas(df)
        return TextDataController(ds,**kwargs)
    
    
    def _map_dset(self,dset,func):
        if self.is_streamed:
            return dset.map(func,
                            batched=self.is_batched,
                            batch_size=self.batch_size
                           )
        return dset.map(func,
                        batched=self.is_batched,
                        batch_size=self.batch_size,
                        num_proc=self.num_proc
                       )
    
    def _filter_dset(self,dset,func):
        if self.is_streamed:
            return dset.filter(func,
                            batched=self.is_batched,
                            batch_size=self.batch_size
                           )
        return dset.filter(func,
                        batched=self.is_batched,
                        batch_size=self.batch_size,
                        num_proc=self.num_proc
                       )
                     
    def validate_input(self):
        if self.is_streamed:
            print('Input validation check is disabled when data is streamed')
            return
        _df = self.dset.to_pandas()
        check_input_validation(_df)
    
    
    
    def save_as_pickles(self,
                        fname, # Name of the pickle file
                        parent='pickle_files', # Parent folder
                        drop_data_attributes=False # Whether to drop all large-size data attributes
                       ):
        if drop_data_attributes:
            if hasattr(self, 'main_ddict'):
                del self.main_ddict
        save_to_pickle(self,fname,parent=parent)
    
        
    def _check_validation_leaking(self):
        if self.val_ratio is None or self.is_streamed:
            return
        
        trn_txt = self.main_ddict['train'][self.main_text]
        val_txt = self.main_ddict['validation'][self.main_text]        
        val_txt_leaked = check_text_leaking(trn_txt,val_txt)
        
        if len(val_txt_leaked)==0: return
        
        # filter train dataset to get rid of leaks
        print('Filtering leaked data out of training set...')
        _func = partial(lambda_batch,
                        feature=self.main_text,
                        func=lambda x: x.strip().lower() not in val_txt_leaked,
                        is_batched=self.is_batched)
        self.main_ddict['train'] = self._filter_dset(self.main_ddict['train'],_func)   
        print('Done')
           
    def _train_test_split(self):
        print_msg('Train Test Split',20)
        val_key = list(set(self.ddict_rest.keys()) & set(['val','validation','valid']))
        if len(val_key)==1: # val split exists
            print('Validation split already exists')
            self.main_ddict=DatasetDict({'train':self.dset,
                                         'validation':self.ddict_rest.pop(val_key[0])})
            
    
        elif self.val_ratio is None: # use all data
            print('No validation split defined')
            self.main_ddict=DatasetDict({'train':self.dset})
        
        elif isinstance(self.val_ratio,list) or isinstance(self.val_ratio,np.ndarray): # filter with indices
            print('Validation indices are provided')
            if self.is_streamed: raise ValueError('Data streaming does not support validation set filtering using indices')
            val_idxs = list(self.val_ratio)
            trn_idxs = list(set(range(len(self.dset))) - set(val_idxs))
            self.main_ddict=DatasetDict({'train':self.dset.select(trn_idxs),
                                         'validation':self.dset.select(val_idxs)})
            
        elif (isinstance(self.val_ratio,float) or isinstance(self.val_ratio,int)) and not len(self.stratify_cols):
            print('Validation split based on val_ratio')
            if self.is_streamed:
                # shuffle dataset before splitting it
                self.dset = self.dset.shuffle(seed=self.seed,buffer_size=self.buffer_size)
                if isinstance(self.val_ratio,float):
                    warnings.warn("Length of streamed dataset is unknown to use float validation ratio. Default to 5000 data points for validation")
                    self.val_ratio=5000
                    
                trn_dset = self.dset.skip(self.val_ratio)
                val_datas = list(self.dset.take(self.val_ratio))
                val_dict={k: [v[k] for v in val_datas] for k in val_datas[0].keys()}
                val_dset = Dataset.from_dict(val_dict)
                self.main_ddict=DatasetDict({'train':trn_dset,
                                         'validation':val_dset})
#                 self.main_ddict=DatasetDict({'train':self.dset.skip(self.val_ratio),
#                                          'validation':self.dset.take(self.val_ratio)})
            else:
                # train val split
                self.main_ddict = self.dset.train_test_split(test_size=self.val_ratio,shuffle=True,seed=self.seed)
                self.main_ddict['validation']=self.main_ddict['test']
                del self.main_ddict['test']
        
        else: # val_ratio split with stratifying
            if self.is_streamed: raise ValueError('Stratified split is not supported for streamed data')                
            if self.is_multilabel and self.label_names[0] in self.stratify_cols:
                raise ValueError('For MultiLabel classification, you cannot choose the label as your stratified column')
            print('Validation split based on val_ratio, with stratifying')
            # Create a new feature 'stratified', which is a concatenation of values in stratify_cols
            if self.is_batched:
                stratified_creation = lambda x: {'stratified':
                                     ['_'.join(list(map(str,[x[v][i] for v in self.stratify_cols]))) 
                                      for i in range(len(x[self.stratify_cols[0]]))]}
            else:
                stratified_creation = lambda x: {'stratified':
                                     '_'.join(list(map(str,[x[v] for v in self.stratify_cols]))) 
                                      }
            self.dset = self.dset.map(stratified_creation,
                                      batched=self.is_batched,
                                      batch_size=self.batch_size,
                                      num_proc=self.num_proc)
            self.dset=self.dset.class_encode_column("stratified")
            # train val split
            self.main_ddict = self.dset.train_test_split(test_size=self.val_ratio,
                                                         shuffle=True,seed=self.seed,
                                                        stratify_by_column='stratified')
            self.main_ddict['validation']=self.main_ddict['test']
            del self.main_ddict['test']
            self.main_ddict=self.main_ddict.remove_columns(['stratified'])
            
        
        del self.dset
        print('Done')

                             
    def _create_label_mapping_func(self,encoder_classes):
        if self.is_multihead:
            label2idxs = [{v:i for i,v in enumerate(l_classes)} for l_classes in encoder_classes]
                    
            _func = lambda inp: {'label': [[label2idxs[i][v] for i,v in enumerate(vs)] for vs in zip(*[inp[l] for l in self.label_names])] \
                                    if self.is_batched else [label2idxs[i][v] for i,v in enumerate([inp[l] for l in self.label_names])]
                              }
            
        else:
            label2idx = {v:i for i,v in enumerate(encoder_classes[0])}
            _func = partial(lambda_map_batch,
                           feature=self.label_names[0],
                           func=lambda x: label2idx[x],
                           output_feature='label',
                           is_batched=self.is_batched)
        return _func
        
    def _encode_labels(self):
        print_msg('Label Encoding')
        if len(self.label_names)>1:
            self.is_multihead=True
        
        if self.label_lists is not None and not isinstance(self.label_lists[0],list):
            self.label_lists = [self.label_lists]
        
        # get label of first row
        first_label = self.dset[self.label_names[0]][0] if not self.is_streamed else next(iter(self.dset))[self.label_names[0]]
        if isinstance(first_label,list):
            # This is multi-label. Ignore self.label_names[1:]
            self.label_names = [self.label_names[0]]
            self.is_multihead=False
            self.is_multilabel=True
            
        encoder_classes=[]
        if not self.is_multilabel:
            for idx,l in enumerate(self.label_names):
                if self.label_lists is None:
                    l_encoder = LabelEncoder()
                    _ = l_encoder.fit(self.dset[l])
                    l_classes = list(l_encoder.classes_)
                else:
                    l_classes = sorted(list(self.label_lists[idx]))
                encoder_classes.append(l_classes)
            
            _func = self._create_label_mapping_func(encoder_classes)
                
            self.dset = self._map_dset(self.dset,_func)

            val_key = list(set(self.ddict_rest.keys()) & set(['val','validation','valid']))
            if len(val_key)>1: raise ValueError('Your DatasetDict has more than 1 validation split')
            if len(val_key)==1:
                val_key=val_key[0]
                self.ddict_rest[val_key] = self._map_dset(self.ddict_rest[val_key],_func)
                    
        else:
            # For MultiLabel, we transform the label itself to one-hot (or actually, few-hot)
            if self.label_lists is None:
                l_encoder = MultiLabelBinarizer()
                _ = l_encoder.fit(self.dset[self.label_names[0]])
                l_classes = list(l_encoder.classes_)
            else:
                l_classes = sorted(list(self.label_lists[0]))
            
            encoder_classes.append(l_classes)
            
            l_encoder = MultiLabelBinarizer(classes=encoder_classes[0])
            _ = l_encoder.fit(None)
            _func = partial(lambda_map_batch,
                            feature=self.label_names[0],
                            func=lambda x: l_encoder.transform(x),
                            output_feature='label',
                            is_batched=self.is_batched,
                            is_func_batched=True)
            self.dset = self._map_dset(self.dset,_func)                                                  
            
            val_key = list(set(self.ddict_rest.keys()) & set(['val','validation','valid']))
            if len(val_key)>1: raise ValueError('Your DatasetDict has more than 1 validation dataset')
            if len(val_key)==1:
                val_key=val_key[0]
                self.ddict_rest[val_key] = self._map_dset(self.ddict_rest[val_key],_func)
            
        self.label_lists = encoder_classes
        print('Done')
        
    def _process_metadatas(self,dset,ddict_rest=None):
        if len(self.metadatas)>0:
            print_msg('Metadata Simple Processing & Concatenating to Main Content')
            map_func = partial(concat_metadatas,
                               main_text=self.main_text,
                               metadatas=self.metadatas,
                               process_metas=self.process_metas,
                               is_batched=self.is_batched)
            dset = self._map_dset(dset,map_func)
            if ddict_rest is not None:
                ddict_rest = self._map_dset(ddict_rest,map_func)
            print('Done')
        return dset if ddict_rest is None else (dset,ddict_rest)
            
            
    
    def _simplify_ddict(self):
        print_msg('Dropping unused features',20)
        if self.cols_to_keep is None:
            self.cols_to_keep= [self.main_text] + self.metadatas
            if self.label_names is not None: self.cols_to_keep+=self.label_names
        cols_to_remove = set(self.all_cols) - set(self.cols_to_keep)
        self.main_ddict['train']=self.main_ddict['train'].remove_columns(list(cols_to_remove))
        if 'validation' in self.main_ddict.keys():
            self.main_ddict['validation']=self.main_ddict['validation'].remove_columns(list(cols_to_remove))
        print('Done')
    
    def _do_transformation(self,dset,ddict_rest=None):
        if len(self.content_tfms):
            print_msg('Text Transformation',20)
            for tfm in self.content_tfms:
                print_msg(callable_name(tfm))
                _func = partial(lambda_map_batch,
                               feature=self.main_text,
                               func=tfm,
                               is_batched=self.is_batched)
                dset = self._map_dset(dset,_func)
                if ddict_rest is not None:
                    ddict_rest = self._map_dset(ddict_rest,_func)
            print('Done')
        return dset if ddict_rest is None else (dset,ddict_rest)
 
    def _do_filtering(self,dset,ddict_rest=None):
        if len(self.filter_dict):
            print_msg('Data Filtering',20)
            for f,tfm in self.filter_dict.items():
                print_msg(f'Do {callable_name(tfm)} on {f}')
                _func = partial(lambda_batch,
                                feature=f,
                                func=tfm,
                                is_batched=self.is_batched)
                dset = self._filter_dset(dset,_func)
                if ddict_rest is not None:
                    ddict_rest = self._filter_dset(ddict_rest,_func)
            print('Done')
        return dset if ddict_rest is None else (dset,ddict_rest)
    
    def _upsampling(self):
        if len(self.upsampling_list):
            print_msg('Upsampling data',20)
            results=[]
            for f,tfm in self.upsampling_list:
                print_msg(f'Do {callable_name(tfm)} on {f}')
                _func = partial(lambda_batch,
                                feature=f,
                                func=tfm,
                                is_batched=self.is_batched)
                new_dset = self._filter_dset(self.main_ddict['train'],_func)
                results.append(new_dset)
            # slow concatenation for iterable dataset    
            self.main_ddict['train'] = concatenate_datasets(results+[self.main_ddict['train']])
            print('Done')
      
    def _do_augmentation(self):
        
        if len(self.aug_tfms):
            print_msg('Text Augmentation',20)

            seed_notorch(self.seed)
            if not self.is_streamed:  
#                 self.main_ddict['train'] = self.main_ddict['train'].with_transform(partial(augmentation_helper,
#                                                                        text_name=self.main_text,
#                                                                        func=partial(func_all,functions=self.aug_tfms)))              
                for tfm in self.aug_tfms:
                    print_msg(callable_name(tfm))
                    _func = partial(lambda_map_batch,
                                   feature=self.main_text,
                                   func=tfm,
                                   is_batched=self.is_batched)
                    self.main_ddict['train'] = self._map_dset(self.main_ddict['train'],_func)

            else:
                self.main_ddict['train'] = IterableDataset.from_generator(augmentation_stream_generator,
                                               features = self.main_ddict['train'].features,
                                               gen_kwargs={'dset': self.main_ddict['train'],
                                                           'text_name':self.main_text,
                                                           'func':partial(func_all,functions=self.aug_tfms)
                                                          })
            print('Done')
        
    def _convert_to_iterable(self):
        if not self.is_streamed:
            self.main_ddict['train'] = self.main_ddict['train'].to_iterable_dataset(num_shards=self.num_shards)
            self.is_streamed=True
            
    def _do_train_shuffling(self):
        print_msg('Shuffling train set',20)
        self.main_ddict['train'] = self.main_ddict['train'].shuffle(seed=self.seed, buffer_size=self.buffer_size)
        print('Done')
        
    def do_all_preprocessing(self,shuffle_trn=True): 
        if self._processed_call:
            warnings.warn('Your dataset has already been processed. Returning the previous processed DatasetDict...')
            return self.main_ddict
            
        print_msg('Start Main Text Processing',20)
        
        # Filtering
        self.dset,self.ddict_rest = self._do_filtering(self.dset,self.ddict_rest)
        
        # Process metadatas
        self.dset,self.ddict_rest = self._process_metadatas(self.dset,self.ddict_rest)
        
        # Process labels
        if self.label_names is not None:
            self._encode_labels()
        
        # Content transformation
        self.dset,self.ddict_rest = self._do_transformation(self.dset,self.ddict_rest)
         
        # Train Test Split.
        ### self.main_ddict is created here
        self._train_test_split()
        
        # Dropping unused columns
        self._simplify_ddict()
        
        # Check validation leaking
        self._check_validation_leaking()
        
        ### The rest of these functions applies only to the train dataset
        # Upsampling
        self._upsampling()
        
        # Augmentation
        self._do_augmentation()
           
        # Convert train set to iterable
        self._convert_to_iterable()
        
        # Shuffle train
        if shuffle_trn:
            self._do_train_shuffling()
        
        self._processed_call=True
        
        return self.main_ddict
    
        
    
    
    def do_tokenization(self,
                       tokenizer, # Tokenizer (preferably from HuggingFace)
                       is_split_into_words=False, # Is text split into words or not
                       max_length=None, # pad to model's allowed max length (default is max_sequence_length)
                       trn_size=None, # The number of training data to be tokenized
                      ):
        print_msg('Tokenization',20)
        self.tokenizer = tokenizer
        self.is_split_into_words= is_split_into_words
        self.max_length = max_length
        if trn_size is not None:
            if trn_ratio<1:
                raise ValueError("Length of streamed dataset is unknown to use floating ratio")
            self.main_ddict['train'] = self.main_ddict['train'].take(trn_ratio)
        
        for k in self.main_ddict.keys():
            self.main_ddict[k] = self.main_ddict[k].map(partial(tokenize_function,
                                                                    text_name=self.main_text,
                                                                    tok=tokenizer,
                                                                    is_split_into_words=is_split_into_words,
                                                                    max_length=max_length),
                                                            batched=True, # always true
                                                            batch_size=self.batch_size
                                                           )
        print('Done')
        return self.main_ddict
        
    def process_and_tokenize(self,
                             tokenizer, # Tokenizer (preferably from HuggingFace)
                             is_split_into_words=False, # Is text split into list or not
                             max_length=None, # pad to model's allowed max length (default is max_sequence_length)
                             trn_size=None, # The number of training data to be tokenized
                             shuffle_trn=True, # To shuffle the train set before tokenization
                            ):
        _ = self.do_all_preprocessing(shuffle_trn)
        _ = self.do_tokenization(tokenizer,is_split_into_words,max_length,trn_size)
        
    
    def set_data_collator(self,data_collator):
        self.data_collator = data_collator
        
    
    def prepare_test_dataset_from_csv(self,file_path):
        file_path = Path(file_path)
        ds = load_dataset(str(file_path.parent),
                          data_files=file_path.name,
                          split='train')
        return self.prepare_test_dataset(ds)
    
    def prepare_test_dataset_from_df(self,df,validate=True):
        if validate:
            check_input_validation(df)
        ds = Dataset.from_pandas(df)
        return self.prepare_test_dataset(ds)
        
    def prepare_test_dataset(self,test_dset,do_filtering=False):
        print_msg('Start Test Set Transformation',20)

        # Filtering
        if do_filtering:
            test_dset = self._do_filtering(test_dset)
        
        # Process metadatas
        test_dset = self._process_metadatas(test_dset)
        
        
        # Content transformation
        test_dset = self._do_transformation(test_dset)
        
        # Tokenization
        print_msg('Tokenization',20)
        test_dset = test_dset.map(partial(tokenize_function,
                                          text_name=self.main_text,
                                          tok=tokenizer,
                                          is_split_into_words=is_split_into_words,
                                          max_length=max_length),
                                  batched=True, # always true
                                  batch_size=self.batch_size
                                 )
        return test_dset


In [ ]:
show_doc(TextDataController)

---

### TextDataController

>      TextDataController (inp, main_text:str, label_names=None,
>                          class_names_predefined=None, filter_dict={},
>                          metadatas=[], process_metas=True,
>                          content_transformations=[],
>                          val_ratio:list|float|None=0.2, stratify_cols=[],
>                          upsampling_list={}, content_augmentations=[],
>                          seed=None, is_batched=True, batch_size=1000,
>                          num_proc=4, cols_to_keep=None, buffer_size=10000,
>                          num_shards=64)

Initialize self.  See help(type(self)) for accurate signature.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| inp |  |  | HuggingFace Dataset or DatasetDict |
| main_text | str |  | Name of the main text column |
| label_names | NoneType | None | Names of the label (dependent variable) columns |
| class_names_predefined | NoneType | None | List of names associated with the labels (same index order) |
| filter_dict | dict | {} | A dictionary: {feature: filtering_function_based_on_the_feature} |
| metadatas | list | [] | Names of the metadata columns |
| process_metas | bool | True | Whether to do simple text processing on the chosen metadatas |
| content_transformations | list | [] | A list of text transformations |
| val_ratio | list \| float \| None | 0.2 | Ratio of data for validation set. If given a list, validation set will be chosen based on indices in this list |
| stratify_cols | list | [] | Column(s) needed to do stratified shuffle split |
| upsampling_list | dict | {} | A list of tuple. Each tuple: (feature,upsampling_function_based_on_the_feature) |
| content_augmentations | list | [] | A list of text augmentations |
| seed | NoneType | None | Random seed |
| is_batched | bool | True | Whether to perform operations in batch |
| batch_size | int | 1000 | Batch size, for when is_batched is True |
| num_proc | int | 4 | Number of process for multiprocessing |
| cols_to_keep | NoneType | None | Columns to keep after all processings |
| buffer_size | int | 10000 | For shuffling data |
| num_shards | int | 64 | Number of shards |

## Load data + Basic use case

You can create a `TextDataController` from a csv, pandas DataFrame, or directly from a HuggingFace dataset object. Currently, `TextDataController` is designed for text classification, so you must provide the column name for the label (or multi-label)

We will load a sample data, modified to match a task where you need to determine which category `L1` a comment (`Content`) belongs to 

Dataset source: https://www.kaggle.com/datasets/kavita5/review_ecommerce

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df.shape

(23486, 10)

In [ ]:
df.sample(5)

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
15977,1081,39,Simple yet sexy,"I tried on the xs in store, but ti was a littl...",5,1,11,General Petite,Dresses,Dresses
7548,1081,39,NaN,NaN,3,0,0,General,Dresses,Dresses
10555,1022,36,Great fit!,Ag jeans are the only brand that fit my body t...,5,1,4,General,Bottoms,Jeans
21986,1011,31,NaN,NaN,4,1,0,General,Bottoms,Skirts
20537,877,26,Nice top,"I bought this top in slate. it is very nice, l...",4,1,0,General Petite,Tops,Knits


You can create a `TextDataController` from a dataframe. This also provides a quick input validation check (NaN check and Duplication check)

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                )

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


You can also create a `TextDataController` directly from the csv file. The good thing about using HuggingFace Dataset as the main backend of the TextDataController is that you can utilize lots of its useful functionality, such as caching

In [ ]:
tdc = TextDataController.from_csv('sample_data/Womens_Clothing_Reviews.csv',
                                  main_text='Review Text',
                                  label_names='Department Name',
                                 )

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


You can also create a `TextDataController` from a HuggingFace Dataset

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
dset

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


Dataset({
    features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
    num_rows: 23486
})

In [ ]:
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         seed=42
                        )

In the "Input Validation Precheck" above, we notice that our dataset has missing values in the text field and the label field. For now, let's load the data as a Pandas' DataFrame, perform some cleaning, and create our `TextDataController`

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df = df[(~df['Review Text'].isna()) & (~df['Department Name'].isna())].reset_index(drop=True)

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                )

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title    2966
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 1 rows


At this point you can start perform 2 important steps on your data

1. Text preprocessings, Label Encoding, Train/Validation Split
2. Tokenization

We haven't provided any preprocessings to the `TextDataController`; we will see more on how to use preprocessings (step by step) as we progress. In fact, we can even perform NaN filtering as a preprocessing step inside `TextDataController`

In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 2, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done


In [ ]:
ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label'],
        num_rows: 4526
    })
})

Our DatasetDict now has two split: train and validation. Note that train split is now IterableDataset, for processing efficiency

In [ ]:
ddict['validation'][:3]

{'Review Text': ["The fun colors drew me to this but it sure fit weird. the top was fine but it became a bit tent-like in the waist. the material doesn't feel great either.",
  "I was disappointed by this dress. i listened to the reviews and ordered one size up, a size 6 instead of my usual 4, and that was the correct size. fit like a glove. unfortunately, the style left much to be desired. this dress is very costume-y. i felt like a wench from the renaissance fair. not a good vibe. it is a beautiful dress, i'm not denying that, but not as a dress you would actually wear to an event. there's just too much going on with the tapestry look and the many types of fabric. i a",
  "I love this comfy top, and if it couldn't get any better, the pleated back makes it extra special! it is easy to thrown on and still feel put together. i purchased this top in the green and i'm absolutely enthralled by the rich color! i do agree that the quality of the pleats could be slightly better, however i bel

In [ ]:
for i,v in enumerate(ddict['train']):
    print(v)
    if i==2: break

{'Review Text': 'I wanted to love this as i love holding horses but it fits weird, slender in arms but baggy in the torso. i will be returning', 'Department Name': 'Tops', 'label': 4}
{'Review Text': "It is soooo cute!!! i'm trying to figure a few things before deciding to keep..... how one goes to the bathroom when wearing it? i can't figure out how to get it on without help tying the straps in the back?", 'Department Name': 'Bottoms', 'label': 0}
{'Review Text': "I really love the look of this jumpsuit, casual but classy. shapely but comfortable. i'm 5'4 and, similar to another reviewer, proportional torso to leg ratio. the legs were longer on me than the model in the picture, but no problem there because they're cuffed.however the arm holes are too big. my bra shows, and the particular one i got came damaged with some of the embroidery coming unraveled on the back. it's also supposed to be dry cleaned and that's annoying for casual wear. returning it.", 'Department Name': 'Bottoms',

Now we can start with the tokenization

In [ ]:
from transformers import RobertaTokenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
ddict = tdc.do_tokenization(tokenizer,max_length=512)

-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

In [ ]:
print(ddict['validation'][0]['input_ids'][:60])

[0, 133, 1531, 8089, 4855, 162, 7, 42, 53, 24, 686, 2564, 7735, 4, 5, 299, 21, 2051, 53, 24, 1059, 10, 828, 10178, 12, 3341, 11, 5, 13977, 4, 5, 1468, 630, 75, 619, 372, 1169, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(next(iter(ddict['train']))['input_ids'][:60])

[0, 100, 770, 7, 657, 42, 25, 939, 657, 1826, 8087, 53, 24, 10698, 7735, 6, 28701, 11, 3701, 53, 3298, 4740, 11, 5, 28762, 4, 939, 40, 28, 3357, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In `TextDataController`, you can also perform Text Processing and Tokenization with one method

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                )

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title    2966
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 1 rows


In [ ]:
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


You can access the DatasetDict from the instance variable `main_ddict`

In [ ]:
tdc.main_ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

This DatasetDict is ready to be put into any HuggingFace text model.

## Filtering

This preprocessing step allow you to filter out certain values of a certain column in your dataset. Let's say I want to filter out any None value in the column 'Review Text'

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
df[(~df['Review Text'].isna())].isna().sum()

Clothing ID                   0
Age                           0
Title                      2966
Review Text                   0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                13
Department Name              13
Class Name                   13
dtype: int64

We will provide a dictionary containing the name of the column and the filtering function to apply on that column. Note that **the filtering function will receive an item from the column, and the function should return a boolean**

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None},
                                 seed=42
                                )

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4529
    })
})

In [ ]:
for i in tdc.main_ddict['train']:
    assert i['Review Text'] is not None

for i in tdc.main_ddict['validation']['Review Text']:
    assert i is not None

We can even add multiple filtering functions. Remember from our precheck, there are also None values in our label 'Department Name'. Let's filter them out as well. While we at it, let's filter out any rating that is less than 3 (just to showcase what our filtering can do)

In [ ]:
df.Rating.value_counts()

Rating
5    13131
4     5077
3     2871
2     1565
1      842
Name: count, dtype: int64

Note that `TextDataController` will only keep the text, the labels and the metadatas columns; any other column will be dropped. To keep the 'Rating', we need to define the `cols_to_keep` argument

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                              'Rating': lambda x: x>=3
                                             },
                                 cols_to_keep=['Review Text','Rating','Department Name'],
                                 seed=42
                                )

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

----- Do <lambda> on Rating -----


...

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/16206 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4052 [00:00<?, ? examples/s]

Done


In [ ]:
for i in tdc.main_ddict['validation']['Department Name']:
    assert i is not None
    
for i in tdc.main_ddict['validation']['Rating']:
    assert i >= 3

## Metadatas concatenation

If we think the metadatas can be helpful, we can concatenate them into the front of your text, so that our text classification model is aware of it.

In this example, Let's add 'Title' as our metadata

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 metadatas='Title',
                                 process_metas=True, #to preprocess the metadata (currently it's just empty space stripping and lowercasing),
                                 seed=42
                                )

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


...

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
next(iter(tdc.main_ddict['train']))['Review Text']

"cute top . I wasn't excited enough about this to purchase it, but i did like it. i am wearing a medium in the photos and for reference my measurements are 38-30-40."

In [ ]:
tdc.main_ddict['validation']['Review Text'][0] # The metadata for this text is None

' . Such a fun jacket! great to wear in the spring or to the office as an alternative to a formal blazer. very comfortable!'

You can add multiple metadatas. Let's say 'Division Name' is the second metadata.

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 metadatas=['Title','Division Name'],
                                 process_metas=True,
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


...

Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 0, which is 0.00% of training set
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
next(iter(tdc.main_ddict['train']))['Review Text']

"general . cute top . I wasn't excited enough about this to purchase it, but i did like it. i am wearing a medium in the photos and for reference my measurements are 38-30-40."

## Label Encodings

We have briefly gone through the simplest case of label encodings, where we only need to predict 1 single label. We call this **single head classification**

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-eadbd93dc0a62969_*_of_00004.arrow


-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


All label names will be saved in instance variable `label_lists`

In [ ]:
tdc.label_lists

[['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

... and all labels will be encoded

In [ ]:
tdc.main_ddict['validation']['label'][:5]

[2, 4, 4, 1, 1]

We also keep the original labeling, for references

In [ ]:
tdc.main_ddict['validation']['Department Name'][:5]

['Intimate', 'Tops', 'Tops', 'Dresses', 'Dresses']

What if we need to predict 2 different labels as once (we will call this **multi-head classification**). For example, let's define our dataset so that we need to predict both 'Department Name' and 'Division Name'

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names=['Division Name','Department Name'],
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-eadbd93dc0a62969_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9ff90c1ed8061e58_*_of_00004.arrow


-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.label_lists

[['General', 'General Petite', 'Initmates'],
 ['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

We can see that we have two lists, one for label names of 'Division Name', and one for label names of 'Department Name'

In [ ]:
tdc.main_ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Review Text', 'Division Name', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 4526
    })
})

In [ ]:
tdc.main_ddict['validation']['Review Text'][:5]

['Such a fun jacket! great to wear in the spring or to the office as an alternative to a formal blazer. very comfortable!',
 'I thought this shirt was really feminine and elegant. only downsides is some of the punched out holes had fabric still attached which you have cut off with scissors- otherwise the shirt will snag. and the second issue of bigger importance are the low armholes. lots of bra showing- not really sure how to get around that so i always wear it with a cardigan. but it would be nice not to have to. \r\nother than that it looks nice and pairs nicely with almost anything.',
 'This top has a bit of a retro flare but so adorable on. looks really cute with a pair of faded boot cut jeans.',
 'I first spotted this on an retailer employee, she paired it with a peasant top & wore it open w/jeans & boots- so darn cute. love how this peice transitions from summer to fall. i\'m 5\'4" so i had to order the small petite which is perfect. note that this dress is very long! it\'s just

In [ ]:
print(tdc.main_ddict['validation']['Division Name'][:5])
print(tdc.main_ddict['validation']['Department Name'][:5])

['General Petite', 'General Petite', 'General', 'General Petite', 'General Petite']
['Intimate', 'Tops', 'Tops', 'Dresses', 'Dresses']


In [ ]:
tdc.main_ddict['validation']['label'][:5]

[[1, 2], [1, 4], [0, 4], [1, 1], [1, 1]]

Lastly, let's define a **multi-label classification**, where a text can have 1 or more label. Our data don't have such labeling, so we will make a new one, just for demonstration.

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df['Department Name'].unique()[:-1] # skip the nan

array(['Intimate', 'Dresses', 'Bottoms', 'Tops', 'Jackets', 'Trend'],
      dtype=object)

In [ ]:
df['Fake Label'] = [np.random.choice(df['Department Name'].unique()[:-1],size=np.random.randint(2,6),replace=False) for _ in range(len(df))]

In [ ]:
df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,Fake Label
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates,"[Tops, Bottoms]"
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses,"[Trend, Tops, Bottoms, Intimate]"
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"[Intimate, Trend, Jackets, Bottoms, Tops]"
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"[Trend, Tops, Bottoms, Intimate]"
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"[Bottoms, Jackets, Trend, Tops]"


You don't have to add any extra argument; the controller will determine whether this is for multilabel classification, based on the format of the label values

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 filter_dict={'Review Text': lambda x: x is not None},
                                 label_names=['Fake Label'],
                                 seed=42,
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.label_lists

[['Bottoms', 'Dresses', 'Intimate', 'Jackets', 'Tops', 'Trend']]

In [ ]:
tdc.main_ddict['validation']['Fake Label'][2]

['Tops', 'Intimate', 'Dresses']

Since this is **multilabel classification**, the label will be one-hot encoded

In [ ]:
tdc.main_ddict['validation']['label'][2]

[0, 1, 1, 0, 1, 0]

If you want to use `TextDataController` for **regression**, you can twist the input argument to achieve what you want. Let's say I want to predict the 'Rating', which is a numerical column

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         cols_to_keep=['Review Text','Rating'], # we add 'Rating' to cols_to_keep to retain it
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-eadbd93dc0a62969_*_of_00004.arrow
Loading cached split indices for dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-e1ec6166c233ad6f.arrow and /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-6826d016807238a2.arrow


-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4529 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Review Text', 'Rating', 'input_ids', 'attention_mask'],
        num_rows: 4529
    })
})

The last thing to do is to rename the label column to `label`, for consistency

In [ ]:
tdc.main_ddict['train'] = tdc.main_ddict['train'].rename_column("Rating", "label")
tdc.main_ddict['validation'] = tdc.main_ddict['validation'].rename_column("Rating", "label")

In [ ]:
print(next(iter(tdc.main_ddict['train'])))

{'Review Text': 'These pants are so cute and comfortable that i ordered each color.  after wearing the black ones one day, they stretched out so badly that i returned the other two colors.  i wish the fabric was thicker with more stretch because the style and cut is really cute. bummer', 'input_ids': [0, 4528, 9304, 32, 98, 11962, 8, 3473, 14, 939, 2740, 349, 3195, 4, 1437, 71, 2498, 5, 909, 1980, 65, 183, 6, 51, 13596, 66, 98, 7340, 14, 939, 1835, 5, 97, 80, 8089, 4, 1437, 939, 2813, 5, 10199, 21, 33997, 19, 55, 4140, 142, 5, 2496, 8, 847, 16, 269, 11962, 4, 741, 22539, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
print(tdc.main_ddict['validation'][0])

{'Review Text': "This picture doesn't do the skirt justice. i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt. it is really pretty and flattering on.", 'label': 5, 'input_ids': [0, 713, 2170, 630, 75, 109, 5, 16576, 2427, 4, 939, 11153, 24, 19, 10, 8633, 1794, 20585, 1055, 25416, 740, 20734, 20020, 23204, 8, 10, 4334, 24693, 196, 7494, 4, 24, 16, 269, 1256, 8, 34203, 15, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

## Content Transformation

This processing allows you to alter the text content in your dataset. All you need to do is having a function, accepting a single string, and return a new, processed string. Note that this transformation will be applied to ALL of your dataset

Let's say we want to normalize our text, because the text might contain some extra spaces between words, or not follow the "single space after a period" rule

In [ ]:
_tmp = "This is a      sentence,which doesn't follow any rule!No single space is provided after period or punctuation marks.    Maybe there are too many spaces!?!   "

In [ ]:
from underthesea import text_normalize

In [ ]:
text_normalize(_tmp)

"This is a sentence , which doesn't follow any rule ! No single space is provided after period or punctuation marks . Maybe there are too many spaces ! ? !"

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_transformations=text_normalize,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-eadbd93dc0a62969_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9ff90c1ed8061e58_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-fae4d983a7f511ce_*_of_00004.arrow


-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Text Transformation --------------------
----- text_normalize -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
next(iter(tdc.main_ddict['train']))['Review Text']

"I wasn't excited enough about this to purchase it , but i did like it . i am wearing a medium in the photos and for reference my measurements are 38-30-40 ."

In [ ]:
tdc.main_ddict['validation']['Review Text'][0]

'Such a fun jacket ! great to wear in the spring or to the office as an alternative to a formal blazer . very comfortable !'

You can chain multiple functions. Let's say after text normalizing, I want to lowercase the text

In [ ]:
str.lower('tHis IS NoT lowerCASE')

'this is not lowercase'

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_transformations=[text_normalize,str.lower],
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-eadbd93dc0a62969_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9ff90c1ed8061e58_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-fae4d983a7f511ce_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Text Transformation --------------------
----- text_normalize -----
----- lower -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
next(iter(tdc.main_ddict['train']))['Review Text']

"i wasn't excited enough about this to purchase it , but i did like it . i am wearing a medium in the photos and for reference my measurements are 38-30-40 ."

In [ ]:
tdc.main_ddict['validation']['Review Text'][0]

'such a fun jacket ! great to wear in the spring or to the office as an alternative to a formal blazer . very comfortable !'

You can even perform some complex transformations, such as removing text inside parentheses, or replacing some texts via a pattern (which is doable using regular expression). Let's make an example of such transformations, where we remove text inside parentheses, and convert any hashtag into the string 'hashtag'

In [ ]:
import re

In [ ]:
def process_text(s):
    # Remove texts inside parentheses
    s = re.sub(r'\(.*?\)', '', s)
    
    # Convert any hashtag into the string 'hashtag'
    s = re.sub(r'#\w+', 'hashtag', s)
    
    return s.strip()

In [ ]:
process_text("#Promotions There's no way it works (I checked!), however it surprises me #howonearth #mindblowing")

"hashtag There's no way it works , however it surprises me hashtag hashtag"

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_transformations=process_text,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-eadbd93dc0a62969_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9ff90c1ed8061e58_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-fae4d983a7f511ce_*_of_00004.arrow


-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Text Transformation --------------------
----- process_text -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


## Train/Validation Split

There are several ways to perform a train/validation split with `TextDataController`

The first way is when you already have a validation split in your HuggingFace's Dataset. Let's use the Dataset built-in function `train_test_split` to simulate this

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
ddict_with_val = dset.train_test_split(test_size=0.1)
# This will create a 'test' split instead of 'validation', so we will process a bit to have a validation split
ddict_with_val['validation']=ddict_with_val['test']
del ddict_with_val['test']

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


In [ ]:
ddict_with_val

DatasetDict({
    train: Dataset({
        features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
        num_rows: 21137
    })
    validation: Dataset({
        features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
        num_rows: 2349
    })
})

In [ ]:
tdc = TextDataController(ddict_with_val,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/21137 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/2349 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/20376 [00:00<?, ? examples/s]

...

Done
-------------------- Train Test Split --------------------
Validation split already exists
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 2, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/20364 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/2264 [00:00<?, ? examples/s]

Done


In [ ]:
tdc.main_ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2264
    })
})

A second way is to split randomly based on a ratio (a float between 0 and 1), or based on the number of data in your validation set

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         val_ratio=0.15,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)
print(tdc.main_ddict)

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-eadbd93dc0a62969_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9ff90c1ed8061e58_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-fae4d983a7f511ce_*_of_00004.arrow


-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 2, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/19233 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/3395 [00:00<?, ? examples/s]

Done
DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3395
    })
})


In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         val_ratio=5000,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)
print(tdc.main_ddict)

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-eadbd93dc0a62969_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9ff90c1ed8061e58_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-fae4d983a7f511ce_*_of_00004.arrow


-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 4, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/17628 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Done
DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Review Text', 'Department Name', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5000
    })
})


A third way is to do a random stratified split (inspired by [sklearn's](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)). Let's do a stratified split based on our label 'Department Name'

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df['Department Name'].value_counts(normalize=True)

Department Name
Tops        0.445978
Dresses     0.269214
Bottoms     0.161852
Intimate    0.073918
Jackets     0.043967
Trend       0.005070
Name: proportion, dtype: float64

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols='Department Name',
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


...

Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 2, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
pd.Series([v['Department Name'] for v in tdc.main_ddict['train']]).value_counts(normalize=True)

Tops        0.444033
Dresses     0.271602
Bottoms     0.161878
Intimate    0.072983
Jackets     0.044309
Trend       0.005193
Name: proportion, dtype: float64

In [ ]:
pd.Series(tdc.main_ddict['validation']['Department Name']).value_counts(normalize=True)

Tops        0.444101
Dresses     0.271542
Bottoms     0.161732
Intimate    0.073133
Jackets     0.044189
Trend       0.005303
Name: proportion, dtype: float64

You can also use multiple columns for your stratification

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols=['Department Name','Rating'],
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


...

Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 2, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


And finally, you can omit any validation split if you specify `val_ratio` as ```None```

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=None,
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)
tdc.main_ddict

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


Map (num_proc=4):   0%|          | 0/22628 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
No validation split defined
Done
-------------------- Dropping unused features --------------------
Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------
Done


DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
})

## Upsampling

This is useful when you have an imbalanced dataset and you want to perform some upsampling (oversampling) on the minority class. In `TextDataController`, you can perform upsampling on any column of the original dataset, and you can even do upsampling on multiple columns at once

Behind the scene, upsampling contains 2 steps; first, the subset of the data is collected based on the filtering condition, and then this subset is concatenated back into the original data

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df['Department Name'].sample(frac=0.8).value_counts()

Department Name
Tops        8356
Dresses     5044
Bottoms     3079
Intimate    1377
Jackets      820
Trend        100
Name: count, dtype: int64

In [ ]:
df['Department Name'].sample(frac=0.8).value_counts(normalize=True)

Department Name
Tops        0.444368
Dresses     0.273768
Bottoms     0.160320
Intimate    0.072650
Jackets     0.043622
Trend       0.005273
Name: proportion, dtype: float64

Let's say I want to upsampling the 'Trend' by the factor of 2 (x2 the amount of 'Trend' data)

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols='Department Name',
                                 upsampling_list=[('Department Name',lambda x: x=='Trend')],
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


...

Done
-------------------- Upsampling data --------------------
----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/18100 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
pd.Series([v['Department Name'] for v in tdc.main_ddict['train']]).value_counts()

Tops        8037
Dresses     4916
Bottoms     2930
Intimate    1321
Jackets      802
Trend        188
Name: count, dtype: int64

The percenntage of 'Trend' data in the train set has approximately doubled (note that we filter some NaN text value so the result is not exactly doubled)

In [ ]:
pd.Series(tdc.main_ddict['validation']['Department Name']).value_counts(normalize=True)

Tops        0.444101
Dresses     0.271542
Bottoms     0.161732
Intimate    0.073133
Jackets     0.044189
Trend       0.005303
Name: proportion, dtype: float64

Since augmentation is applied only to the train set, the distribution of label in the validation set remains the same

Similarly, you can triple the amount of 'Trend' by repeating the procedure twice. In the following examples, I will triple the 'Trend' and double the 'Jackets'

In [ ]:
tdc = TextDataController.from_df(df,
                                 main_text='Review Text',
                                 label_names='Department Name',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols='Department Name',
                                 upsampling_list=[('Department Name',lambda x: x=='Trend'),
                                                  ('Department Name',lambda x: x=='Trend'),
                                                  ('Department Name',lambda x: x=='Jackets')
                                                 ],
                                 # This can be simplified as
#                                  upsampling_list=[('Department Name',lambda x: x=='Trend' or x=='Jackets'),
#                                                   ('Department Name',lambda x: x=='Trend')],
                                 seed=42
                                )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

----- Input Validation Precheck -----
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
----- Label Encoding -----


...

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/18100 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
pd.Series([v['Department Name'] for v in tdc.main_ddict['train']]).value_counts()

Tops        8037
Dresses     4916
Bottoms     2930
Jackets     1604
Intimate    1321
Trend        282
Name: count, dtype: int64

**A word of warning**: **Upsampling is a slow procedure**, as it requires multiple dataset concatenation. This procedure is can be extremely slow for streamed dataset

## Content Augmentation

Similarly to Content Transformation, Content Augmentation allows to alter the text content in your dataset. You also need to provide a function accepting a single string, and return a new, processed string. Unlike Content Transformation which is applied to ALL data, the Content Augmentation only applies to your TRAINING data

One of the popular library for data augmentation is [nlpaug](https://github.com/makcedward/nlpaug). We will demonstrate how to integrate its augmentation functions into our `TextDataController`

In [ ]:
import nlpaug.augmenter.char as nac

In [ ]:
_tmp = "I like my clothes loose fitting but even for me this ran large, i am 5'7 134b and medium fit in the shoulders but was too big overall"

In [ ]:
# augmentation by replacing character with nearby one on the keyboard
def nlp_aug(x,aug=None):
    return aug.augment(x)[0]

In [ ]:
aug = nac.KeyboardAug(aug_char_max=3,aug_char_p=0.1,aug_word_p=0.07)
nearby_aug_func = partial(nlp_aug,aug=aug)

In [ ]:
nearby_aug_func(_tmp)

"I like my clothes loose fittibg but even for me this ran large, i am 5 ' 7 134b and Jedium fit in the shoulders but was too big overal."

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_augmentations=nearby_aug_func,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-eadbd93dc0a62969_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9ff90c1ed8061e58_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-fae4d983a7f511ce_*_of_00004.arrow
Loading cached split indices for dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc8

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done


Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-badfb725db509bf2_*_of_00004.arrow


- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...
Done
-------------------- Text Augmentation --------------------
----- nlp_aug -----


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
for i,v in enumerate(tdc.main_ddict['train']):
    if i==4: break
    print(v['Review Text'])
    print('-'*20)
    

I wasn ' t excited eno6gh about this to purchase it, but i did like it. i am weariGg a medium in the (hotos and for reference my measurements are 38 - 30 - 40.
--------------------
I oTdered the manYo and the navy in size 28. i am 5 ' and 130 lbs. both arrives with the size 28 label on them, the mango fit perfedtly. perfect length on the legs and very flattering. the navy, on the other hand, was an inch shorter and an inch narrower! i Oept douHle - checking the packing slip and the label inside the shorts to see if they accidentally sent a 27p. but no, it was clearly a 28. i brought that back to the store and ordered another naFy 28, hoping that )air was just mislabeled 28 or
--------------------
TNis shirt looked plain online, but when i see it in stoTe, i thought it ' s so pretty! it is the over - sized sty<e, when i tried it on, it ' s so flattering, even though it is over - sized but it doesn ' t make you look chunky at all! i got it while it was on sale wity extra 25% off! i rang$

Again, since this is Content Augmentation, the validation set is unmodified.

In [ ]:
tdc.main_ddict['validation']['Review Text'][0]

'Such a fun jacket! great to wear in the spring or to the office as an alternative to a formal blazer. very comfortable!'

You can even apply Content Augmentation stochastically, by adding a random condition in your augmentation function

In [ ]:
aug = nac.KeyboardAug(aug_char_max=3,aug_char_p=0.1,aug_word_p=0.07)

In [ ]:
def nlp_aug_stochastic(x,aug=None,p=0.5):
    return aug.augment(x)[0] if random.random()<p else x
nearby_aug_func = partial(nlp_aug_stochastic,aug=aug,p=0.5)

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_augmentations=nearby_aug_func,
                         seed=42
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-eadbd93dc0a62969_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9ff90c1ed8061e58_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-fae4d983a7f511ce_*_of_00004.arrow
Loading cached split indices for dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc8

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
----- Do <lambda> on Department Name -----
Done
----- Label Encoding -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done


Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-f893627565d98cd2/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-badfb725db509bf2_*_of_00004.arrow


- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...
Done
-------------------- Text Augmentation --------------------
----- nlp_aug_stochastic -----


Map (num_proc=4):   0%|          | 0/18099 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
-------------------- Tokenization --------------------


Map:   0%|          | 0/4526 [00:00<?, ? examples/s]

Done


In [ ]:
for i,v in enumerate(tdc.main_ddict['train']):
    if i==5: break
    print(v['Review Text'])
    print('-'*20)
    

I wasn ' t excited enougG about this to purchase it, but i did liue it. i am wearing a medJum in the photos and for reference my measurements are 38 - 30 - 40.
--------------------
I ordered the mango and the navy in size 28. i am 5' and 130 lbs. both arrived with the size 28 label on them, the mango fit perfectly. perfect length on the legs and very flattering. the navy, on the other hand, was an inch shorter and an inch narrower! i kept double-checking the packing slip and the label inside the shorts to see if they accidentally sent a 27p. but no, it was clearly a 28. i brought that back to the store and ordered another navy 28, hoping that pair was just mislabeled 28 or
--------------------
This shirt looked plain online, but when i see it in store, i thought it ' s so pretGy! it is the over - sized style, whFn i triev it on, it ' s so flattering, evem though it is over - s8zed but it doesn ' t make you look shunky at all! i got it while it was on sale with extra 25% off! i range fr

One of the advanced augmentation is "Contexttual Word Embeddings Augmenter" (https://github.com/makcedward/nlpaug/blob/master/example/textual_augmenter.ipynb), where you can insert/substitute words using LLM such as BERT, RoBERTA ...

In [ ]:
import nlpaug.augmenter.word as naw

In [ ]:
aug = naw.ContextualWordEmbsAug(model_path='roberta-base', 
#                                 device='cuda:0', # uncomment this if you have GPU
                                action="substitute",
                               aug_p=0.07)

In [ ]:
_tmp = "I like my clothes loose fitting but even for me this ran large, i am 5'7 134b and medium fit in the shoulders but was too big overall"

In [ ]:
contextual_aug_func = partial(nlp_aug,aug=aug)

In [ ]:
contextual_aug_func(_tmp)

"I like my clothes loose fitting but even for me this sounds large, i scale 5'7 134b and medium fit in the shoulders but am too big overall"

Technically, you can put `contextual_aug_func` into the `content_augmentations` argument, as shown below, and it will run, but this takes forever since it will augment line-by-line (this is the default procedure for augmentation `TextDataController`), which means it doesn't utilize the batching options of nlpaug. If the demain for this functionality is large enough, I will consider adding a special case for nlpaug's contextual augmentation in `TextDataController`

```python
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataController(dset,
                         main_text='Review Text',
                         label_names='Department Name',
                         filter_dict={'Review Text': lambda x: x is not None,
                                      'Department Name': lambda x: x is not None,
                                     },
                         content_augmentations=contextual_aug_func,
                         seed=42,
                         is_batched=False,
                         num_proc=1 # set this to 1 when running augmentation on GPU
                        )
tdc.process_and_tokenize(tokenizer,max_length=512,shuffle_trn=True)
```

## Streaming Capability

## Let's go

In [ ]:
# main_ddict = load_dataset('secret_data',data_files=['buyer_listening_with_all_raw_data_w28.csv','buyer_listening_with_all_raw_data_w28.csv'],split='train')
# main_ddict

# main_ddict = load_dataset('sample_data',data_files=['sample_large.csv','sample_large.csv','sample_large.csv'])
# main_ddict

main_ddict = load_dataset('sample_data',data_files=['sample_large.csv'],split='train')
main_ddict

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/sample_data-96e446a75e3f09ba/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


Dataset({
    features: ['Source', 'Content', 'L1', 'L2'],
    num_rows: 2269
})

In [ ]:
main_ddict[:5]

{'Source': ['Google Play', 'Non Owned', 'Google Play', 'Owned', 'Google Play'],
 'Content': ['App ncc lúc nào cx lag đơ, phần tìm kiếm thì viết kiểu gì sp đó vẫn ko ra, thế phải ghi đúng tên mới chịu à? Lỡ quên tên ngta ghi mé mé như v cx phải gợi ý sp tương tự chứ??? ☻',
  '..❗️ GÓC THANH LÝ Tính ra rẻ hơn cả mua #Shopee Mong 1 lần đc check ib mỏi tay 😆😆😆   Em chuyển cửa hàng nên dọn lại có thừa vài chục tấm nệm xuất nhật này.   1mx2m : 1m2x2 : 1m4x2m : 1m6x2m : 1m8x2m :2mx2m Đệm dày 7-8 phân Nhưng vì còn ít nên topic này em bán thanh lý giá rẻ ạ.   Ship cod nhận hàng được kiểm tra thoải mái.  Miễn ship toàn quốc. Nên đừng bom tội nghiệp em nhé dày 7-8 phân Nhắn tin em gửi mẫu nhé🥰',
  'Mắc gì người ta đặt hàng toàn lỗi 😃????',
  '#GhienShopeePayawardT8 Khi bạn chơi shopee quá lâu thì không thể nào không biết đến với ShopeePay . Liên kết thanh toán được cho các đơn hàng Shopee và ShopeeFood luôn nè.',
  'Rất bức xúc khi dùng . mã giảm giá người dùng thì m02 vậy cho ưu đãi đấy làm gì ạ

In [ ]:
%%time
main_ddict[0]

CPU times: user 210 µs, sys: 118 µs, total: 328 µs
Wall time: 227 µs


{'Source': 'Google Play',
 'Content': 'App ncc lúc nào cx lag đơ, phần tìm kiếm thì viết kiểu gì sp đó vẫn ko ra, thế phải ghi đúng tên mới chịu à? Lỡ quên tên ngta ghi mé mé như v cx phải gợi ý sp tương tự chứ??? ☻',
 'L1': 'Feature',
 'L2': 'App performance'}

In [ ]:
seed_everything(42)

In [ ]:
_filter_dict={'L1':lambda x: x!='Others'}

_content_tfms = partial(apply_vnmese_word_tokenize,normalize_text=True)
_content_tfms.__name__='VNM word segmentation'

_upsampling_dict={
    'Source': lambda x: x=='hc search' if random.random()<0.5 else False
}

_aug_tfms=partial(remove_vnmese_accent,prob=0.5)

In [ ]:
tdc = TextDataController(main_ddict,main_text='Content',
                         label_names='L1',
                         filter_dict=_filter_dict,
                         metadatas='Source',
                         content_transformations=_content_tfms,
                         val_ratio=0.25,
                         stratify_cols='Source',
                         upsampling_dict=_upsampling_dict,
                         content_augmentations=_aug_tfms,
                         seed=42,
                         is_batched=True,
                         is_streamed=False
                        )

In [ ]:
%%time
my_ddict = tdc.do_all_preprocessing(shuffle_trn=True)
# 2x big data
# is_batched True, shuffle_trn True. Wall time: 1min 6s
# is_batched False, shuffle_trn True, Wall time: 1min 18s

# 3x sample_large.csv
# is_batched True, shuffle_trn True. Wall time: 2.79 s
# is_batched False, shuffle_trn True, Wall time: 2.96 s

Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-96e446a75e3f09ba/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-e8025282b10ede62_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-96e446a75e3f09ba/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-0ecebfc0c7f83a1d_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-96e446a75e3f09ba/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-966fc97b537cb573_*_of_00004.arrow
Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/sample_data-96e446a75e3f09ba/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-9067e41b077d8eb9_*_of_00004.arrow


-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on L1 -----
Done
----- Metadata Simple Processing & Concatenating to Main Content -----
Done
----- Label Encoding -----
Done
-------------------- Text Transformation --------------------
----- VNM word segmentation -----
Done
-------------------- Train Test Split --------------------


Map (num_proc=4):   0%|          | 0/1458 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/1458 [00:00<?, ? examples/s]

Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.09% of training set
Filtering leaked data out of training set...


...

Done
-------------------- Upsampling data --------------------
----- Do <lambda> on Source -----


Filter (num_proc=4):   0%|          | 0/1091 [00:00<?, ? examples/s]

Done
-------------------- Text Augmentation --------------------
----- remove_vnmese_accent -----


Map (num_proc=4):   0%|          | 0/1112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling train set --------------------
Done
CPU times: user 252 ms, sys: 154 ms, total: 406 ms
Wall time: 736 ms


In [ ]:
my_ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Source', 'Content', 'L1'],
        num_rows: 365
    })
})

In [ ]:
%%time
my_ddict['validation'][0]

CPU times: user 478 µs, sys: 0 ns, total: 478 µs
Wall time: 310 µs


{'Source': 'non owned',
 'Content': 'non owned . Mã_500K toàn sàn Shopee cho ai cần nè » https://shope.ee/10QzJtpqQi',
 'L1': 1}

In [ ]:
_tmp=iter(my_ddict['train'])

In [ ]:
%%time
next(_tmp)

CPU times: user 28.2 ms, sys: 301 µs, total: 28.5 ms
Wall time: 26.8 ms


{'Source': 'google play',
 'Content': 'google play . Chán ơi là chánnnn Toàn_bị U02_Mn có biết cách nào chữa ko thì chỉ em với ạ 🥰',
 'L1': 3}

In [ ]:
%%time
next(_tmp)

CPU times: user 63 µs, sys: 53 µs, total: 116 µs
Wall time: 118 µs


{'Source': 'google play',
 'Content': 'google play . Ứng_dụng quá là đơ . Càng dùng càng đơ thì mua_bán cái mẹ j .',
 'L1': 3}

In [ ]:
tokenizer


PhobertTokenizer(name_or_path='vinai/phobert-base', vocab_size=64000, model_max_length=256, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True)

In [ ]:
tdc.main_ddict

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Source', 'Content', 'L1'],
        num_rows: 365
    })
})

In [ ]:
# tdc.main_ddict['validation'] = tdc.main_ddict['validation'].to_iterable_dataset(num_shards=tdc.num_shards)

In [ ]:
my_ddict_tok = tdc.do_tokenization(tokenizer)

Map:   0%|          | 0/365 [00:00<?, ? examples/s]

In [ ]:
my_ddict_tok

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Source', 'Content', 'L1', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 365
    })
})

In [ ]:
%%time
_ = my_ddict_tok['validation'][0]

CPU times: user 287 µs, sys: 140 µs, total: 427 µs
Wall time: 380 µs


In [ ]:
my_ddict_tok['validation'][0]['Content']

'non owned . Mã_500K toàn sàn Shopee cho ai cần nè » https://shope.ee/10QzJtpqQi'

In [ ]:
print(tokenizer.convert_ids_to_tokens(my_ddict_tok['validation'][0]['input_ids'])[:100])

['<s>', 'non', 'ow@@', 'ned', '.', 'Mã_@@', '500@@', 'K', 'toàn', 'sàn', 'Sho@@', 'pee', 'cho', 'ai', 'cần', 'nè', '»', 'htt@@', 'ps@@', '://@@', 'sho@@', 'pe@@', '.@@', 'ee@@', '/@@', '10@@', 'Q@@', 'z@@', 'J@@', 't@@', 'p@@', 'q@@', 'Q@@', 'i', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
_iter=iter(my_ddict_tok['train'])

In [ ]:
%%time
_tmp = next(_iter)

CPU times: user 272 ms, sys: 123 µs, total: 272 ms
Wall time: 272 ms


In [ ]:
%%time
_tmp = next(_iter)

CPU times: user 0 ns, sys: 5 µs, total: 5 µs
Wall time: 6.68 µs


In [ ]:
print(_tmp['Content'])

print(tokenizer.convert_ids_to_tokens(_tmp['input_ids'])[:100])

google play . App nhu cc ko lam gi cung xoa tk
['<s>', 'google', 'play', '.', 'App', 'nhu', 'cc', 'ko', 'lam', 'gi', 'cung', 'xoa', 't@@', 'k', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
i2v = {i:v for i,v in enumerate(tdc.label_lists[0])}

vc_l1 = pd.Series(my_ddict['validation']['L1']).value_counts(normalize=True).reset_index()

In [ ]:
vc_l1['index'] = vc_l1['index'].map(i2v)
vc_l1
# Feature                    0.371056
#  Commercial                 0.209191
#  Delivery                   0.137174
#  Shopee account             0.127572
#  Buyer complained seller    0.043896
#  Return/Refund              0.030864
#  Payment                    0.030178
#  Order/Item                 0.028121
#  Services                   0.021948

,index,proportion
0,Feature,0.369863
1,Commercial,0.213699
2,Shopee account,0.131507
3,Delivery,0.128767
4,Buyer complained seller,0.041096
5,Return/Refund,0.032877
6,Services,0.032877
7,Order/Item,0.027397
8,Payment,0.021918


In [ ]:
pd.Series(my_ddict['validation']['Source']).value_counts(),pd.Series(my_ddict['validation']['Source']).value_counts(normalize=True)
#  Google Play    0.743484
#  Non Owned      0.093278
#  Owned          0.069959
#  iOS            0.061043
#  HC search      0.032236

(google play    271
 non owned       34
 owned           26
 ios             22
 hc search       12
 Name: count, dtype: int64,
 google play    0.742466
 non owned      0.093151
 owned          0.071233
 ios            0.060274
 hc search      0.032877
 Name: proportion, dtype: float64)

In [ ]:
_tmp_trn = list(my_ddict['train'])

In [ ]:
len(my_ddict['validation'])

365

In [ ]:
# len(_tmp_trn) + len(my_ddict['validation'])

In [ ]:
len(_tmp_trn)

1106

In [ ]:
int((47-12)*0.5) + (1458 - 365)

1110

## Let's stream

In [ ]:
# main_ddict = load_dataset('secret_data',data_files=['buyer_listening_with_all_raw_data_w28.csv','buyer_listening_with_all_raw_data_w28.csv'],split='train')
# main_ddict

# main_ddict = load_dataset('sample_data',data_files=['sample_large.csv','sample_large.csv','sample_large.csv'],streaming=True)
# main_ddict

main_ddict = load_dataset('sample_data',data_files=['sample_large.csv'],split='train',streaming=True)
main_ddict

<datasets.iterable_dataset.IterableDataset>

In [ ]:
# for i,v in enumerate(main_ddict):
#     print(v)
#     if i==4: break

In [ ]:
seed_everything(42)


In [ ]:
labels=['Feature','Commercial','Delivery',
        'Shopee account','Buyer complained seller',
        'Return/Refund','Payment','Order/Item',
        'Services','Others']


In [ ]:
_filter_dict={'L1':lambda x: x!='Others'}

_content_tfms = partial(apply_vnmese_word_tokenize,normalize_text=True)
_content_tfms.__name__='VNM word segmentation'

_upsampling_dict={
    'Source': lambda x: x=='hc search' if random.random()<0.5 else False
}

_aug_tfms=partial(remove_vnmese_accent,prob=0.85)

In [ ]:
main_ddict = load_dataset('sample_data',data_files=['sample_large.csv'],split='train',streaming=True)
tdc = TextDataController(main_ddict,main_text='Content',
                         label_names='L1',
                         class_names_predefined=labels,
                         filter_dict=_filter_dict,
                         metadatas='Source',
                         content_transformations=_content_tfms,
                         val_ratio=365, 
#                          upsampling_dict=_upsampling_dict, # super slow
                         content_augmentations=_aug_tfms,
                         seed=42,
                         is_batched=True,
                         is_streamed=True,
                         num_shards=512
                        )

In [ ]:
import gc
gc.collect()

70

In [ ]:
%%time
my_ddict = tdc.do_all_preprocessing(shuffle_trn=True)
# 2x big data
# is_batched True, shuffle_trn True. Wall time: 1min 6s
# is_batched False, shuffle_trn True, Wall time: 1min 18s

# 3x sample_large.csv
# is_batched True, shuffle_trn True. Wall time: 2.79 s
# is_batched False, shuffle_trn True, Wall time: 2.96 s

# TODO: redo
# 3x sample_large.csv, streaming, with aug
# is_batched True, shuffle_trn True. Wall time: 53.5 s
# is_batched False, shuffle_trn True, Wall time: 17.4 s



-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on L1 -----
Done
----- Metadata Simple Processing & Concatenating to Main Content -----
Done
----- Label Encoding -----
Done
-------------------- Text Transformation --------------------
----- VNM word segmentation -----
Done
-------------------- Train Test Split --------------------
Done
-------------------- Dropping unused features --------------------
Done
-------------------- Text Augmentation --------------------
Done
-------------------- Shuffling train set --------------------
Done
CPU times: user 2.62 s, sys: 2.64 s, total: 5.26 s
Wall time: 5.26 s


In [ ]:
my_ddict['validation']

Dataset({
    features: ['Source', 'Content', 'L1'],
    num_rows: 365
})

In [ ]:
my_ddict_tok = tdc.do_tokenization(tokenizer)

Map:   0%|          | 0/365 [00:00<?, ? examples/s]

In [ ]:
my_ddict_tok

DatasetDict({
    train: <datasets.iterable_dataset.IterableDataset object>
    validation: Dataset({
        features: ['Source', 'Content', 'L1', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 365
    })
})

In [ ]:
# len(list(my_ddict_tok['train'])),len(list(my_ddict_tok['validation']))


In [ ]:
_iter=iter(my_ddict_tok['train'])
_tmp = next(_iter)
_tmp = next(_iter)

In [ ]:
len(my_ddict_tok['validation'])

365

In [ ]:
_tmp=iter(my_ddict['train'])

In [ ]:
%%time
next(_tmp)
# 9.44s with upsampling
# 4.72s without upsampling
# 5.16 without upsampling, with augmentation

CPU times: user 2.29 s, sys: 2.93 s, total: 5.22 s
Wall time: 5.22 s


{'Source': 'google play',
 'Content': 'google play . Bi loi khong hien_hinh_anh , xoa di tai lai app deu ko hien_thi gi het .',
 'L1': 3}

In [ ]:
%%time
next(_tmp)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 8.58 µs


{'Source': 'hc search',
 'Content': 'hc search . khong cap_nhat duoc sdt',
 'L1': 9}

In [ ]:
_tmp=iter(my_ddict['validation'])

In [ ]:
%%time
next(_tmp)
# 9.44s

CPU times: user 173 µs, sys: 154 µs, total: 327 µs
Wall time: 241 µs


{'Source': 'google play',
 'Content': 'google play . đang chơi mà quảng_cáo',
 'L1': 1}

In [ ]:
%%time
next(_tmp)

CPU times: user 80 µs, sys: 0 ns, total: 80 µs
Wall time: 83 µs


{'Source': 'google play',
 'Content': 'google play . Cứ quản cáo quá nhiều , app nào củng gặp quản_cáo của shopee 😆',
 'L1': 1}

In [ ]:
len(list(my_ddict['train'])),len(list(my_ddict['validation']))
# (1124, 365)

(1093, 365)

In [ ]:
len(list(my_ddict_tok['train'])),len(list(my_ddict_tok['validation']))
# (1124, 365)

(1093, 365)

In [ ]:
for i,v in enumerate(my_ddict['train']):
    print(v)
    if i==9: break

{'Source': 'ios', 'Content': 'ios . Đã chỉnh đi chỉnh lại rất nhiều nhưng nó vẫn ghi là thanh toán ko khả dụng! \nLà sao hả SHOPEE:)))', 'L1': 6}
{'Source': 'hc search', 'Content': 'hc search . kh đc mượt', 'L1': 3}
{'Source': 'google play', 'Content': 'google play . Hóng đơn hàng về .khi về thì Shipper chưa giao mà báo k ai nhận. Tự ý huỷ đơn. Shipper Ngũ Hành Sơn Đà Nẵng quá kém.', 'L1': 2}
{'Source': 'google play', 'Content': 'google play . Có cái nịt thùng mì 50k phí sip 100', 'L1': 2}
{'Source': 'google play', 'Content': 'google play . Alo mới đặt hàng đi vắng có 1 bữa Xong shipper nt chửi ôm xồm vậy Đánh giá 1 sao cho biết', 'L1': 2}
{'Source': 'google play', 'Content': 'google play . Đc', 'L1': 5}
{'Source': 'google play', 'Content': 'google play . Tôi vưa bị đăng xuất khỏi shoppe một cách vô lý. Bây h k vào lại đc. Có vào đc thì tất cả nhưg đơn hàg trc đó của tui cug đã bị mất. Yêu cầu giai quyết vấn đề', 'L1': 9}
{'Source': 'google play', 'Content': 'google play . Giao hàng gì

In [ ]:
_tmp=iter(my_ddict_tok['validation'])

In [ ]:
print(my_ddict_tok['validation']['Content'][0])

print(tokenizer.convert_ids_to_tokens((my_ddict_tok['validation']['input_ids'][0])[:100]))

google play . đang chơi mà quảng_cáo
['<s>', 'google', 'play', '.', 'đang', 'chơi', 'mà', 'quảng_cáo', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


Let's start a step-by-step walkthrough on how to use this class

## Load data

In [ ]:
from datasets import Dataset,load_dataset

In [ ]:
DATA_PATH = Path('sample_data')

In [ ]:
df = pd.read_csv(DATA_PATH/'sample_large.csv')

df.head()

In [ ]:
main_ddict = Dataset.from_csv(str(DATA_PATH/'sample_large.csv'))
main_ddict

In [ ]:
# main_ddict = load_dataset(str(DATA_PATH),data_files={'train':'sample_large.csv'})
main_ddict = load_dataset(str(DATA_PATH),data_files='sample_large.csv')
main_ddict

In [ ]:
main_ddict = load_dataset(str(DATA_PATH),data_files='sample_large.csv',split='train')
main_ddict

In [ ]:
main_ddict = load_dataset(str(DATA_PATH),data_files='sample_large.csv',split='train',streaming=True)
main_ddict

<datasets.iterable_dataset.IterableDataset>

In [ ]:
next(iter(main_ddict))

{'Source': 'Google Play',
 'Content': 'App ncc lúc nào cx lag đơ, phần tìm kiếm thì viết kiểu gì sp đó vẫn ko ra, thế phải ghi đúng tên mới chịu à? Lỡ quên tên ngta ghi mé mé như v cx phải gợi ý sp tương tự chứ??? ☻',
 'L1': 'Feature',
 'L2': 'App performance'}

In [ ]:
list(main_ddict.take(3))

In [ ]:
%%time
main_ddict = load_dataset(str(DATA_PATH),data_files=['sample_large.csv','sample_large.csv'])
main_ddict

In [ ]:
# main_ddict = load_dataset('csv',data_files=str(DATA_PATH/'sample_large.csv'))
# main_ddict

In [ ]:
_tmp = Path('secret_data')/'some_files.csv'

In [ ]:
_tmp.parent,_tmp.name

(Path('secret_data'), 'some_files.csv')

In [ ]:
_tmp = Path('some_files.csv')
_tmp.parent

Path('.')

In [ ]:
%%time
main_ddict = load_dataset('secret_data',data_files='buyer_listening_with_all_raw_data_w28.csv',split='train')
main_ddict

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/secret_data-042d1badc74881bf/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


CPU times: user 1.06 ms, sys: 3.97 ms, total: 5.03 ms
Wall time: 4.48 ms


Dataset({
    features: ['Week', 'Group', 'Source', 'Content', 'L1', 'L2', 'L3', 'L4', 'is_valid', 'iteration'],
    num_rows: 114605
})

In [ ]:
%%time
main_ddict = load_dataset('csv',data_files='secret_data/buyer_listening_with_all_raw_data_w28.csv',split='train')
main_ddict

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/quan/.cache/huggingface/datasets/csv/default-042d1badc74881bf/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.
CPU times: user 592 ms, sys: 84.5 ms, total: 677 ms
Wall time: 1.72 s


Dataset({
    features: ['Week', 'Group', 'Source', 'Content', 'L1', 'L2', 'L3', 'L4', 'is_valid', 'iteration'],
    num_rows: 114605
})

## Actual data loading

In [ ]:
main_ddict = load_dataset('secret_data',data_files='buyer_listening_with_all_raw_data_w28.csv')
if hasattr(main_ddict,'keys'):
    print('yes')
print(main_ddict.keys())

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/secret_data-042d1badc74881bf/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

yes
dict_keys(['train'])


In [ ]:
main_ddict

DatasetDict({
    train: Dataset({
        features: ['Week', 'Group', 'Source', 'Content', 'L1', 'L2', 'L3', 'L4', 'is_valid', 'iteration'],
        num_rows: 114605
    })
})

In [ ]:
_tmp = main_ddict.pop('train')

In [ ]:
main_ddict

DatasetDict({
    
})

In [ ]:
main_ddict = load_dataset('secret_data',data_files='buyer_listening_with_all_raw_data_w28.csv',split='train')
main_ddict

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/secret_data-042d1badc74881bf/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


Dataset({
    features: ['Week', 'Group', 'Source', 'Content', 'L1', 'L2', 'L3', 'L4', 'is_valid', 'iteration'],
    num_rows: 114605
})

In [ ]:
if hasattr(main_ddict,'keys'):
    print('yes')

## Process metadatas

In [ ]:
main_ddict

Dataset({
    features: ['Week', 'Group', 'Source', 'Content', 'L1', 'L2', 'L3', 'L4', 'is_valid', 'iteration'],
    num_rows: 114605
})

In [ ]:
main_ddict[-3:]

{'Week': [28.0, 28.0, 28.0],
 'Group': ['Tú Bà Bà',
  'Gia Lai-Thanh lý đồ dùng và thời trang',
  'thuonghieucongluan.com.vn'],
 'Source': ['Non Owned', 'Non Owned', 'Non Owned'],
 'Content': ['Riết rồi k biết xài cái quần gì để k bị trừ tiền oan. Bữa trc thì vụ nạp thẻ qua shopee T chửi chưa đã cái miệng. Đổi qua momo cho lành. Ỷ y momo k có số dư nên chắc k sao. Chủ yếu lk tk ngân hàng thôi, cái nó tự liên kết thẻ trừ giao dịch qua apple mỗi tuần 129k/ tuần (ông cố ơi!)🙂Thêm cái chỉnh ảnh 419k/ năm (cái này cũng được đi).Dm bữa h tự trừ hết hơn 1tr trong tk ngân hàng. Ui là trời. 2 hộp sữa của con T ra đi nữa rồi đó.Hiện đại, hại điện.Khỏi cảm ơn, T xoá app rồi.Dm trả lại T 2 hộp Meiji đi rồi T sử dụng lại.!',
  'GHN có còn ship cho Sendo không các thím? Em mua shopee, lazada, TIKI thì đơn do best, GHN vs viettel ship nhiều nhất mà mấy bên này chắc ship hầu hết các sàn lớn đúng ko nhờ?',
  'Cục QLTT Hà Nội: Kiểm tra, xử lý nhiều vụ hàng lậu, hàng giả'],
 'L1': ['Feature', 'Delivery',

In [ ]:
metadatas = ['Week','Source']
# metadatas = val2iterable(metadatas)
process_metas = True
main_text='Content'
is_batched=True

In [ ]:
main_ddict = load_dataset('secret_data',data_files='buyer_listening_with_all_raw_data_w28.csv',split='train')
main_ddict

Found cached dataset csv (/home/quan/.cache/huggingface/datasets/csv/secret_data-042d1badc74881bf/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


Dataset({
    features: ['Week', 'Group', 'Source', 'Content', 'L1', 'L2', 'L3', 'L4', 'is_valid', 'iteration'],
    num_rows: 114605
})

In [ ]:
# def _process_metadatas(ds:dict,
#                        main_text,
#                        metadatas,
#                        process_metas=True,
#                        sep='.',
#                       is_batched=True):
#     metadatas = val2iterable(metadatas)
#     results={main_text:ds[main_text]}
#     for m in metadatas:
#         m_data = ds[m]
#         if process_metas:
#             # just strip and lowercase
#             m_data = [str(v).strip().lower() for v in m_data] if is_batched else str(m_data).strip().lower()
#         results[m]=m_data
#         if is_batched:
#             results[main_text] = [f'{m_data[i]}{sep} {results[main_text][i]}' for i in range(len(m_data))]
#         else:
#             results[main_text] = f'{m_data}{sep} {results[main_text]}'
#     return results


In [ ]:
print_msg('Metadatas Simple Processing & Concatenating to Main Content')
main_ddict_meta = main_ddict.map(partial(_process_metadatas,
                                         main_text=main_text,
                                         metadatas=metadatas,
                                         process_metas=process_metas,
                                         is_batched=is_batched),
                                batched=is_batched)

----- Metadatas Simple Processing & Concatenating to Main Content -----


Map:   0%|          | 0/114605 [00:00<?, ? examples/s]

In [ ]:
main_ddict_meta[:3]

{'Week': [1.0, 1.0, 1.0],
 'Group': ['Google Play', 'Google Play', 'Google Play'],
 'Source': ['google play', 'google play', 'google play'],
 'Content': ['google play. 1.0. Tại sao cứ hiện thông báo',
  'google play. 1.0. Mlem',
  'google play. 1.0. 1 số sản phẩm trong giỏ hàng vừa đc cập nhật trong khi giỏ ko còn 1 hàng nào nx 😀'],
 'L1': ['Services', 'Others', 'Feature'],
 'L2': ['Shopee communication channels', 'Cannot defined', 'Cart & Order'],
 'L3': ['Annoying pop-up ads', '-', 'Cart issues/suggestions'],
 'L4': ['Non-tech', '-', 'Tech'],
 'is_valid': [None, None, None],
 'iteration': [1, 1, 1]}

In [ ]:
print_msg('Metadata Simple Processing & Concatenating to Main Content')
main_ddict_meta = main_ddict.map(partial(_process_metadatas,
                                         main_text=main_text,
                                         metadatas=metadatas,
                                         process_metas=process_metas,
                                         is_batched=False),
                                batched=False)

Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/secret_data-042d1badc74881bf/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-cadaba33433fe0f4.arrow


----- Metadata Simple Processing & Concatenating to Main Content -----


In [ ]:
main_ddict_meta[:3]

{'Week': [1.0, 1.0, 1.0],
 'Group': ['Google Play', 'Google Play', 'Google Play'],
 'Source': ['google play', 'google play', 'google play'],
 'Content': ['google play. 1.0. Tại sao cứ hiện thông báo',
  'google play. 1.0. Mlem',
  'google play. 1.0. 1 số sản phẩm trong giỏ hàng vừa đc cập nhật trong khi giỏ ko còn 1 hàng nào nx 😀'],
 'L1': ['Services', 'Others', 'Feature'],
 'L2': ['Shopee communication channels', 'Cannot defined', 'Cart & Order'],
 'L3': ['Annoying pop-up ads', '-', 'Cart issues/suggestions'],
 'L4': ['Non-tech', '-', 'Tech'],
 'is_valid': [None, None, None],
 'iteration': [1, 1, 1]}

In [ ]:
# stream
main_ddict_stream = load_dataset('secret_data',data_files='buyer_listening_with_all_raw_data_w28.csv',split='train',streaming=True)
main_ddict_stream

<datasets.iterable_dataset.IterableDataset>

In [ ]:
main_ddict_meta = main_ddict_stream.map(partial(_process_metadatas,
                                         main_text=main_text,
                                         metadatas=metadatas,
                                         process_metas=process_metas,
                                         is_batched=True),
                                batched=True)

In [ ]:
list(main_ddict_meta.take(3))

[{'Week': '1.0',
  'Group': 'Google Play',
  'Source': 'google play',
  'Content': 'google play. 1.0. Tại sao cứ hiện thông báo',
  'L1': 'Services',
  'L2': 'Shopee communication channels',
  'L3': 'Annoying pop-up ads',
  'L4': 'Non-tech',
  'is_valid': None,
  'iteration': 1},
 {'Week': '1.0',
  'Group': 'Google Play',
  'Source': 'google play',
  'Content': 'google play. 1.0. Mlem',
  'L1': 'Others',
  'L2': 'Cannot defined',
  'L3': '-',
  'L4': '-',
  'is_valid': None,
  'iteration': 1},
 {'Week': '1.0',
  'Group': 'Google Play',
  'Source': 'google play',
  'Content': 'google play. 1.0. 1 số sản phẩm trong giỏ hàng vừa đc cập nhật trong khi giỏ ko còn 1 hàng nào nx 😀',
  'L1': 'Feature',
  'L2': 'Cart & Order',
  'L3': 'Cart issues/suggestions',
  'L4': 'Tech',
  'is_valid': None,
  'iteration': 1}]

In [ ]:
main_ddict_meta = main_ddict_stream.map(partial(_process_metadatas,
                                         main_text=main_text,
                                         metadatas=metadatas,
                                         process_metas=process_metas,
                                         is_batched=False),
                                batched=False)
# batched does not matter when using streamed
list(main_ddict_meta.take(3))

[{'Week': '1.0',
  'Group': 'Google Play',
  'Source': 'google play',
  'Content': 'google play. 1.0. Tại sao cứ hiện thông báo',
  'L1': 'Services',
  'L2': 'Shopee communication channels',
  'L3': 'Annoying pop-up ads',
  'L4': 'Non-tech',
  'is_valid': None,
  'iteration': 1},
 {'Week': '1.0',
  'Group': 'Google Play',
  'Source': 'google play',
  'Content': 'google play. 1.0. Mlem',
  'L1': 'Others',
  'L2': 'Cannot defined',
  'L3': '-',
  'L4': '-',
  'is_valid': None,
  'iteration': 1},
 {'Week': '1.0',
  'Group': 'Google Play',
  'Source': 'google play',
  'Content': 'google play. 1.0. 1 số sản phẩm trong giỏ hàng vừa đc cập nhật trong khi giỏ ko còn 1 hàng nào nx 😀',
  'L1': 'Feature',
  'L2': 'Cart & Order',
  'L3': 'Cart issues/suggestions',
  'L4': 'Tech',
  'is_valid': None,
  'iteration': 1}]

In [ ]:
main_ddict_meta

Dataset({
    features: ['Week', 'Group', 'Source', 'Content', 'L1', 'L2', 'L3', 'L4', 'is_valid', 'iteration'],
    num_rows: 114605
})

## Process labels

In [ ]:
metadatas = ['Week','Source']
process_metas = True
main_text='Content'
is_batched=True

In [ ]:
print_msg('Metadatas Simple Processing & Concatenating to Main Content')
main_ddict_meta = main_ddict.map(partial(_process_metadatas,
                                         main_text=main_text,
                                         metadatas=metadatas,
                                         process_metas=process_metas,
                                         is_batched=is_batched),
                                batched=is_batched)

Loading cached processed dataset at /home/quan/.cache/huggingface/datasets/csv/secret_data-042d1badc74881bf/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-ac679a0b1f06a8e9.arrow


----- Metadatas Simple Processing & Concatenating to Main Content -----


In [ ]:
len(main_ddict_meta['L1'])

114605

In [ ]:
#     def _encode_labels(self):
#         print_msg('Label Encoding')
#         if self.label_names is None: 
#             raise ValueError('Missing label columns!')
#         self.label_names = val2iterable(self.label_names)
#         if len(self.label_names)>1:
#             self.is_multihead=True
        
#         if self.label_lists is not None and not isinstance(self.label_lists[0],list):
#             self.label_lists = [self.label_lists]
        
#         if isinstance(self.df[self.label_names[0]].iloc[0],list):
########               (self.dset[self.label_names[0]][0],list)
#             # This is multi-label. Ignore self.label_names[1:]
#             self.label_names = [self.label_names[0]]
#             self.is_multihead=False
#             self.is_multilabel=True
            
#         encoder_classes=[]
#         if not self.is_multilabel:
#             for idx,l in enumerate(self.label_names):
#                 if self.label_lists is None:
#                     train_label = self.df[l].values
#                     l_encoder = LabelEncoder()
#                     self.df[l] = l_encoder.fit_transform(train_label)
#                     encoder_classes.append(list(l_encoder.classes_))
#                 else:
#                     l_classes = sorted(list(self.label_lists[idx]))
#                     label2idx = {v:i for i,v in enumerate(l_classes)}
#                     self.df[l] = self.df[l].map(label2idx).values
#                     encoder_classes.append(l_classes)
#         else:
#             # For MultiLabel, we only save the encoder classes without transforming the label itself to one-hot (or actually, few-hot)
#             if self.label_lists is None:
#                 l_encoder = MultiLabelBinarizer()
#                 _ = l_encoder.fit(self.df[self.label_names[0]])
#                 encoder_classes.append(list(l_encoder.classes_))
#             else:
#                 l_classes = sorted(list(self.label_lists[0]))
#                 encoder_classes.append(l_classes)
                
#         self.label_lists = encoder_classes

## Constructor/ Class Method calls

If you just want to get the dataframe from the csv path, set ```return_df=True```. You still have the input validation precheck functionality.

In [ ]:
df = TextDataMain.from_csv(DATA_PATH/'sample_large.csv',
                            return_df=True)

The ```Input Validation Precheck``` will check for missing values and duplicate rows in the csv file. Since there's no such thing in our sample dataset, we won't see anything here

In [ ]:
df.sample(5)

In [ ]:
df.Source.value_counts()

Let's say you are happy with this dataframe (after you did some others preprocessing), then you can start creating a `TextDataMain` object

For this dataframe, I want to 
- Build a text classification model, with main text in ```Content``` column, metadatas is ```Source```, and the label is ```L1```
- Perform `apply_word_tokenize` with text normalization (this is "text transformation")
- For augmentation, I want to perform: Oversampling the ```Owned, Non Owned and HC Search``` from column ```Source```, then add some the Vietnamese no-accent text. Note that all of these are called "text augmentation"

Let's define these transformations

> For Text Transformation

In [ ]:
awt_tfm = partial(apply_word_tokenize,normalize_text=True)
# You can also set a __name__ to your augmentation function. 
# This way you will have meaningful text messages as outputs
awt_tfm.__name__='UTS Word Tokenization With Normalization'

txt_tfms=[awt_tfm]

> For Text Augmentation

In [ ]:
# apply_to_all means I will apply this augmentation to all the data 
# (including the original data and the augmented data/transformed data from previous augmentation/transformation)
over_nonown_tfm = partial(sampling_with_condition,query='Source=="non owned"',frac=0.5,seed=42,apply_to_all=False)
over_nonown_tfm.__name__ = 'Oversampling Non Owned'

over_own_tfm = partial(sampling_with_condition,query='Source=="owned"',frac=2,seed=42,apply_to_all=False)
over_own_tfm.__name__ = 'Oversampling Owned'

over_hc_tfm = partial(sampling_with_condition,query='Source=="hc search"',frac=2.5,seed=42,apply_to_all=False)
over_hc_tfm.__name__ = 'Oversampling HC search'

remove_accent_tfm = partial(remove_vnmese_accent,frac=1,seed=42,apply_to_all=True)
remove_accent_tfm.__name__ = 'Add No-Accent Text'

aug_tfms = [over_nonown_tfm,over_own_tfm,over_hc_tfm,remove_accent_tfm]


In [ ]:
tdm = TextDataMain(df,
                    main_content='Content',
                    metadatas='Source', # You can put a list of multiple metadatas
                    label_names='L1', # You can put a list of multiple labels
                    val_ratio=0.2,
                    split_cols='L1', # You can even put a list of multiple columns to be used for validation splitting
                    content_tfms = txt_tfms, # You can add multiple content transformation functions ...
                    aug_tfms = aug_tfms, # ... as well as augmentation functions
                    process_metadatas=True,
                    seed=42,
                    shuffle_trn=True)

If we want to directly create a ```TextDataMain``` object from our csv file, we can instead use this:

In [ ]:
tdm = TextDataMain.from_csv(DATA_PATH/'sample_large.csv',
                            return_df=False,
                            main_content='Content',
                            metadatas='Source',
                            label_names='L1',
                            val_ratio=0.2,
                            split_cols='L1',
                            content_tfms = txt_tfms,
                            aug_tfms = aug_tfms,
                            process_metadatas=True,
                            seed=42,
                            shuffle_trn=True)

In [ ]:
show_doc(TextDataMain.to_df)

Note that all the previous constructor calls do not do any heavy processing yet.

To actually run all the processes, one can call `TextDataMain.to_df()`

In [ ]:
df_processed = tdm.to_df()

Notice this?
```
Previous Validation Percentage: 20.0%
- Before leak check
Size: 14
- After leak check
Size: 14
- Number of rows leaked: 0, or 0.00% of the original validation (or test) data
Current Validation Percentage: 20.0%
```
After performing train/test split, the ```TextDataMain``` object also perform a "leak check": After `text_transformation` is performed, it will compare the text from ```Content``` value in the validation set to the ```Content``` text in the train set. Any duplications (texts that belong to both set) will be removed from validation set.

In [ ]:
df_processed.sample(5)

Note that, since we have metadatas, the metadatas is concatenated to the front of the texture content

In [ ]:
df_processed.Content.sample(5).values

We now have a new dataframe with only the necessary columns (the processed text column, metadatas, label, and ```is_valid``` which tells you which row belongs to the validation set). Notice that our class has also encode our label for us

Our TextDataMain object also stores other useful attributes, such as:

In [ ]:
# The entire processed dataframe, similar to the df_processed above
tdm.df.head()

In [ ]:
# class names (This will be a list of list, as this class can handle multi-label classification)
tdm.label_lists

In [ ]:
# a dictionary storing unique value for each provided metadata
tdm.metadata_dict

If we want to see how a HuggingFace's tokenizer work on our processed text:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

In [ ]:
# this will pick a random text from train set to show
tdm.tokenizer_explain_single(tokenizer)

By doing this, we can see how the tokenizer interact with our text.

In [ ]:
show_doc(TextDataMain.to_datasetdict)

Since we need to convert our data to HuggingFace's DatasetDict format in order to utilize HuggingFace's model well, we can directly export datasetdict using `TextDataMain.to_datasetdict`

In [ ]:
ddict_sample = tdm.to_datasetdict(tokenizer)

In [ ]:
ddict_sample

In [ ]:
ddict_sample['train']['text'][0]

In [ ]:
print(ddict_sample['train']['input_ids'][0])

Note that PhoBert will auto-pad our sentence to its model max_sequence_length, which is 256

In [ ]:
len(ddict_sample['train']['input_ids'][0])

In [ ]:
ddict_sample['train']['label'][0]

In [ ]:
show_doc(TextDataMain.save_as_pickles)

As the transformations/augmentations can take time for large dataset, we want to save our TextDataMain object. We can use `TextDataMain.save_as_pickles` to export a pickle file

In [ ]:
tdm.save_as_pickles('my_tdm')

Then you can load it with

In [ ]:
tdm2 = TextDataMain.from_pickle('my_tdm')

... and access all the attributes

In [ ]:
tdm2.df.head()

In [ ]:
tdm2.label_lists[0]

In [ ]:
tdm2.metadata_dict

Let's check the file size

In [ ]:
file_stats = os.stat(Path('pickle_files/my_tdm.pkl'))
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

As it saves the entire processed dataframe (and datasetdict if you call ```to_datasetdict```), the pickle size can be large. In some scenario you don't need to store these data attributes (as inference time, or in production). Thus one can save a lighter pickle file by setting ```drop_data_attributes``` to ```True```

In [ ]:
tdm.save_as_pickles('my_lightweight_tdm',drop_data_attributes=True)

In [ ]:
file_stats = os.stat(Path('pickle_files/my_lightweight_tdm.pkl'))
print(f'File Size in MegaBytes is {file_stats.st_size / (1024 * 1024)}')

We will see a bigger file size reduction when we work with much larger dataset

In [ ]:
tdm_light = TextDataMain.from_pickle('my_lightweight_tdm')

You can still access some important attributes (except for any data attributes, such as ```df``` or ```main_ddict```

In [ ]:
tdm_light.label_lists[0]

In [ ]:
tdm_light.metadata_dict

In [ ]:
#| hide
# import nbdev; nbdev.nbdev_export()